In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\nsthtz\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


In [36]:
using DelimitedFiles

function initialiseData()
    full_data = readdlm(joinpath("../../results/neuralnet/pianoroll/neuralnet0.csv"),',')
    full_data = round.(Int, full_data)
    
    return full_data
end

full_data = initialiseData()

128×33 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0   

# # Convert piano roll into timestepx12 tonal mapping


In [37]:
function getActiveNotes(timestep)
    return findall(x->x==1, full_data[:,timestep])
end

function createTonalMap()
    tonal_map = zeros(Int64, size(full_data, 2), 12)

    for i in 1:size(full_data, 2)
        active_notes = getActiveNotes(i)
        for note in active_notes
            tonal_map[i, note%12] += 1
        end
    end

    return tonal_map
end

tonal_map = createTonalMap()

33×12 Array{Int64,2}:
 1  0  0  0  0  0  0  0  1  0  0  0
 0  2  0  0  0  0  0  0  0  0  1  0
 1  0  0  1  0  0  0  0  1  0  0  0
 0  2  0  0  0  2  0  0  0  0  0  0
 0  1  0  1  0  0  0  1  0  0  1  0
 1  0  0  0  0  1  0  0  2  0  0  0
 0  0  0  1  0  1  0  0  1  0  1  0
 1  0  0  1  0  0  0  2  0  0  0  0
 1  0  0  0  0  2  0  0  0  0  1  0
 0  0  0  1  0  1  0  0  0  0  2  0
 1  0  0  0  0  2  0  0  0  0  1  0
 0  0  0  1  0  1  0  1  1  0  0  0
 0  1  0  0  1  0  0  2  0  0  0  0
 ⋮              ⋮              ⋮   
 1  0  0  1  0  1  0  0  0  0  1  0
 0  0  0  1  0  2  0  0  0  0  1  0
 1  0  0  1  0  0  0  0  0  1  1  0
 0  0  0  1  0  1  0  0  0  1  1  0
 1  0  1  0  0  1  0  0  1  0  0  0
 1  0  0  0  1  1  0  1  0  0  0  0
 0  0  0  0  1  2  0  0  1  0  0  0
 1  0  0  0  0  2  0  1  0  0  0  0
 2  0  0  0  0  0  0  1  1  0  0  0
 2  0  0  0  0  1  0  1  0  0  0  0
 2  0  0  0  0  1  0  0  0  1  0  0
 1  0  0  0  0  1  0  0  1  1  0  0

# # Pattern matcher to identify the best-fit key and scale from existing notes

In [21]:
#Score In: Tx12 Matrix, pattern. Out: Best position(s)
function score_key(matrix, pattern)
    counter = zeros(Int64, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    println("Key Counter is ", counter)
    
    key_score = zeros(Int64, 12, 1)
    
    for i in 1:size(pattern, 1)
        for j in 1:size(pattern, 1)
            key_score[i] += circshift(pattern, i-1)[j]*counter[j]
        end
    end
    println("Key Score is ", key_score)
    return key_score
end

# Find maximum from list
function findmaximum(list)
    positions = []
    max_value = maximum(list)
    for (i, x) in enumerate(list)
        if x == max_value
            push!(positions, i)
        end
    end
    return positions
end

# Find an optimal scale
function score_tonality(matrix, positions)
    
    counter = zeros(Int64, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    
    tonality_scores = []
    
    for position in positions
        counter_shifted = circshift(counter, (0,-(position-1)))
        println("Position Counter for ", position, " is ", counter_shifted)
        
        major_score = 0
        minor_score = 0
        
        for i in 1:size(counter_shifted, 2)
            major_score += majorpattern[i]*counter_shifted[i]
            minor_score += minorpattern[i]*counter_shifted[i]
        end

        if major_score > minor_score
            push!(tonality_scores, [position, major_score, 1])
        else
            push!(tonality_scores, [position, minor_score, 0])
        end
        
    end 
    
    return tonality_scores
end

score_tonality (generic function with 1 method)

In [38]:
function findScale()
    # Scale patterns
    pattern = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]
    majorpattern = [1, 0, 0.25, 0, 0.25, 1, 0, 1, 0, 0.25, 0, 0.5]
    minorpattern = [0.25, 0, 1, 0, 1, 0.25, 0, 0.25, 0.25, 1, 0, 0.25]


    Keys = Dict{Integer, String}(1 => "C", 2 => "C#", 3 => "D", 4 => "D#", 5 => "E", 6 => "F", 7 => "F#", 8 => "G", 9 => "G#", 10 => "A", 11 => "A#", 12 => "B")

    # Score matrix after pattern
    key_score_result = score_key(tonal_map, pattern)
    # score_result = [9, 5, 18, 9, 23, 18, 8, 2, 7, 16, 12, 11]

    # Find positions of maximum score
    positions = findmaximum(key_score_result)
    sort!(positions)

    # # Use positions to find the best key

    tonality_results = score_tonality(tonal_map, positions)
    println(tonality_results)

    # [Position (1 = C), Score, Scale (1 = Major, 0 = Minor)]
    best_key = [0, 0, 1]

    # In case of tied scores, check each for the best scale
    for result_group in tonality_results
        if best_key[2] < result_group[2]
            best_key[1] = result_group[1]
            best_key[2] = round.(Int, result_group[2])
            best_key[3] = result_group[3]
        end
    end

    #Save Key
    println(best_key)
    if best_key[3] == 1
        key = best_key[1]
        key_print = Keys[best_key[1]]
        scale = "Major"
    else
        key = ( (12+(best_key[1] - 1) - 3)%12 ) + 1
        key_print = Keys[key]
        scale = "Minor"
    end
    println(key, " ", key_print, " ", scale)
    return key, scale
end
    
key, scale = findScale()

Key Counter is [27 6 1 11 5 32 0 12 17 4 13 0]
Key Score is [81; 106; 28; 113; 43; 94; 79; 49; 118; 33; 100; 52]
Position Counter for 9 is [17 4 13 0 27 6 1 11 5 32 0 12]
Any[[9.0, 84.75, 0.0]]
[9, 85, 0]
6 F Minor


(6, "Minor")

# # Define patterns for chord functions relative to key

In [41]:
function initialiseFunctionDefinitions()
    # Function entry: "Name" => ([notes], [Bass note], [Double/Triple notes], [Rhythmical Weight (Strong, Weak, Off)])
    functions = Dict{String, Array{Int64, 1}}

    if scale == "Major"
    #Major  =  [1, 0, 2, 0, 3, 4, 0, 5, 0, 6, 0, 7]
    #

        functions = Dict(
        "T"    =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O", "A"]),
        "S"    =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "D"    =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "D"]),
        "T-5"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O"]),
        "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "T/3"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((4+(key-1))%12)+1, 3) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["S", "W", "O"]),
        "S/3"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 3) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "D/3"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "T/5"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 5) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["W", "O"]),
        "S/5"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 5) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["W", "O"]),
        "D/5"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["W", "O"]),
        "D64"  =>  (circshift([4, 0, 0, 0, 6, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
        "T64"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 0, 0, 6, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
        "T54"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
        "D54"  =>  (circshift([4, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
        "D7"   =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O", "D"]),
        "D7/3" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ nothing ], ["S", "W", "O"]),
        "D7/5" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ nothing ], ["W", "O"]),
        "D7/7" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((5+(key-1))%12)+1, 7) ], [ nothing ], ["S", "W", "O"]),
        "D7-1" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 0, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((5+(key-1))%12)+1, 7) ], ["S", "W", "O"]),
        "D7-5" =>  (circshift([0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "S65"  =>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
        "S65/6"=>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 6) ], [ nothing ], ["S", "W", "O"]),
        "S6"   =>  (circshift([0, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "Ss"   =>  (circshift([0, 0, 1, 0, 0, 3, 0, 0, 0, 5, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 1) ], [ (((2+(key-1))%12)+1, 1), (((5+(key-1))%12)+1, 3) ], ["S", "W", "O"]),
        "Ts"   =>  (circshift([3, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 1) ], [ (key, 3), (((9+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "Tm"   =>  (circshift([0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 5], (key-1)), [ (((4+(key-1))%12)+1, 1) ], [ (((4+(key-1))%12)+1, 1), (((7+(key-1))%12)+1, 3) ], ["S", "W", "O"]))

    elseif scale == "Minor"
    #                         [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11]
    #Minor  =                 [1, 0, 2, 3, 0, 4, 0, 5, 6, 0, 7, 7+]              
        functions = Dict(
        "T"    =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O", "A"]),
        "S"    =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "D"    =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "T-5"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O"]),
        "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "T/3"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((3+(key-1))%12)+1, 3) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["S", "W", "O"]),
        "S/3"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((8+(key-1))%12)+1, 3) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "D/3"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "T/5"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 5) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["W", "O"]),
        "S/5"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 5) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["W", "O"]),
        "D/5"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["W", "O"]),
        "D64"  =>  (circshift([4, 0, 0, 6, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
        "T64"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 0, 6, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
        "T54"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
        "D54"  =>  (circshift([4, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
        "D7"   =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
        "D7/3" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ nothing ], ["S", "W", "O"]),
        "D7/5" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ nothing ], ["W", "O"]),
        "D7/7" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((5+(key-1))%12)+1, 7) ], [ nothing ], ["S", "W", "O"]),
        "D7-1" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 0, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((5+(key-1))%12)+1, 7) ], ["S", "W", "O"]),
        "D7-5" =>  (circshift([0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "S65"  =>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
        "S65/6"=>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 6) ], [ nothing ], ["S"]),
        "S6"   =>  (circshift([0, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "Dm"   =>  (circshift([0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 1, 0], (key-1)), [ (((10+(key-1))%12)+1, 1) ], [ (((10+(key-1))%12)+1, 1), (((2+(key-1))%12)+1, 3) ], ["S", "W", "O"]),
        "Ts"   =>  (circshift([3, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0], (key-1)), [ (((8+(key-1))%12)+1, 1) ], [ (key, 3), (((8+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "Tm"   =>  (circshift([0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 5, 0], (key-1)), [ (((3+(key-1))%12)+1, 1) ], [ (((3+(key-1))%12)+1, 1), (((7+(key-1))%12)+1, 3) ], ["S", "W", "O"]))
    end
        
    return functions
end
    
functions = initialiseFunctionDefinitions()

Dict{String,Tuple{Array{Int64,1},Array{Tuple{Int64,Int64},1},Array{T,1} where T,Array{String,1}}} with 27 entries:
  "S65/6" => ([0, 3, 0, 0, 0, 5, 0, 6, 0, 0, 1, 0], Tuple{Int64,Int64}[(8, 6)],…
  "S/5"   => ([0, 3, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0], Tuple{Int64,Int64}[(6, 5)],…
  "T"     => ([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], Tuple{Int64,Int64}[(6, 1)],…
  "Ts"    => ([0, 1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0], Tuple{Int64,Int64}[(2, 1)],…
  "S/3"   => ([0, 3, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0], Tuple{Int64,Int64}[(2, 3)],…
  "T/5"   => ([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], Tuple{Int64,Int64}[(1, 5)],…
  "D7/5"  => ([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 7, 0], Tuple{Int64,Int64}[(8, 5)],…
  "Dm"    => ([0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 5, 0], Tuple{Int64,Int64}[(4, 1)],…
  "T/3"   => ([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], Tuple{Int64,Int64}[(9, 3)],…
  "D/5"   => ([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], Tuple{Int64,Int64}[(8, 5)],…
  "T-5"   => ([0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], Tuple{Int64,Int64}[(6, 

# # Analyze tonal map and calculate tonal distance from original data for each timestep

In [49]:
function tonal_distance(input, target, double_notes, triple=false, debug=false)
    input = deepcopy(input)
    target = deepcopy(target)
    target = map(x -> x > 0 ? 1 : 0, target)

    double_index = []
    two_doubles = false
            
    if !isnothing(double_notes[1])
        if size(double_notes, 1) == 2
            two_doubles = true
            double_index = [double_notes[1][1], double_notes[2][1]]
        end
        for note in double_notes
            if triple
                target[note[1]] += 2
            else
            target[note[1]] += 1
            end
        end
    end
    
    if debug
        println("Input:  ", input)
        println("Target: ", target)
    end
          
    
    distance = 0

    subvector = target - input
    
    # Subtract input-vector from target vector. Any notes that are in place are removed immediately. The rest are resolved in a left/right iterative BFS search, counting the distance
    # moved until opposite signed note is found for resolution. NB: chords with multiple possible doubled notes are resolved to the closest match, distancewise.
    k = 1
    j = 1

    while any(x->x != 0, subvector)

        i = 6+k
        paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])
        if debug
            println(paddedsub)
            println(i)
        end

        if paddedsub[i] != 0 
            if sign(paddedsub[i-j]) != sign(paddedsub[i]) && paddedsub[i-j] != 0
                
                regular = i-j+6 % 12
                if regular == 0
                    regular = 12
                end
                if debug
                    println("k: ", k)
                    println("i: ", i)
                    println("i-j: ", i-j)
                    println("regular: ", regular)
                end

                if two_doubles && (regular == double_index[1] || regular == double_index[2])

                    
                    for index in double_index
                        subvector[index] -= 1
                    end
                    subvector[k] += 1
                    distance += j
                    
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])

                    two_doubles = false

                    if debug
                        println("Double in i-j+6")
                        println(subvector)
                    end
                elseif two_doubles && (k == double_index[1] || k == double_index[2])

                    
                    for index in double_index
                        subvector[index] -= 1
                    end
                    subvector[regular] += 1
                    distance += j

                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])
                    two_doubles = false

                    if debug
                        println("Double in k for i-j+6")
                        println(subvector)
                    end
                else
                    
                    num = subvector[regular]
                    subvector[regular] = subvector[regular] + (subvector[k] / abs(subvector[k]))
                    subvector[k] = subvector[k] + (num / abs(num))
                    distance += j
                    
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])

                    if debug
                        println("Normal found in i-j")
                        println(subvector)
                    end     
                end

            elseif sign(paddedsub[i+j]) != sign(paddedsub[i]) && paddedsub[i+j] != 0

                

                regular = i+j-6 % 12
                if regular == 0
                    regular = 12
                end
                            
                if debug
                    println("k: ", k)
                    println("i: ", i)
                    println("i+j: ", i+j)
                    println("regular: ", regular)
                end

                if two_doubles && (regular == double_index[1] || regular == double_index[2])
                    
                    for index in double_index
                        subvector[index] -= 1
                    end
                    subvector[k] += 1
                    distance += j
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])

                    two_doubles = false

                    if debug
                        println("Double in i+j-6")
                        println(subvector)
                    end

                elseif two_doubles && (k == double_index[1] || k == double_index[2])

                    
                    for index in double_index
                        subvector[index] -= 1
                    end
                    subvector[regular] += 1
                    distance += j
                                    
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])
                    two_doubles = false

                    if debug
                        println("Double in k for i+j-6")
                        println(subvector)
                    end
                else

                    num = subvector[regular]
                    subvector[regular] = subvector[regular] + (subvector[k] / abs(subvector[k]))
                    subvector[k] = subvector[k] + num / abs(num)  
                    distance += j
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])

                    if debug
                        println("Normal found in i+j")
                        println(subvector)
                    end    
                end
            else
                if debug
                    println("No match found")
                end
                k += 1
            end
        else
            if debug
                println("Initial not found")
            end
            k += 1
        end
        if k == 13
            if debug 
                println("Iteration done for j ", j)
            end
            k = 1
            j += 1
        end
        if j == 7
            if debug
                println("Done")
            end
            break                    
        end
        if debug
            println(paddedsub)
        end
    end
    if debug
        println(subvector)
        println("Distance ", distance)
    end
    return distance
end
               
# Creates sorted and unsorted lists of the tonal distances of each time-step in startindex:finalindex in relation to all available functions
function initialiseFunctionMap(startindex, finalindex)

    # Used for searching best-first
    function_potential = Array{Tuple{Int64,String},1}[] 

    # Used for building shortest path map                            
    function_potential_unsorted = Array{Tuple{Int64,String},1}[] 

    for i in startindex:finalindex
        function_pool = Tuple{Int64, String}[]
        for chord in keys(functions)
            triple=false
            if chord == "T-5" || chord == "S-5" || chord == "D-5"                                                            
                triple = true
            end
            tonal_dist = tonal_distance(tonal_map[i,:], functions[chord][1], functions[chord][3], triple, false)
            function_pool_entry = (tonal_dist, chord)        
            push!(function_pool, (tonal_dist, chord))
        end 
        #println(typeof(function_pool))
        push!(function_potential_unsorted, function_pool)
        push!(function_potential, sort(function_pool))
    end
    return function_potential, function_potential_unsorted
end
       
function_potential, function_potential_unsorted = initialiseFunctionMap(1, 16)          
display(function_potential)

16-element Array{Array{Tuple{Int64,String},1},1}:
 [(0, "D64"), (0, "T"), (0, "T/3"), (0, "T/5"), (0, "Tm"), (1, "D"), (1, "D/3"), (1, "D/5"), (1, "D54"), (1, "D7")  …  (2, "S65/6"), (2, "T54"), (3, "D7-1"), (3, "Dm"), (3, "S"), (3, "S/3"), (3, "S/5"), (3, "T64"), (4, "D-5"), (5, "T-5")]        
 [(2, "D7-5"), (2, "Ts"), (3, "S"), (3, "S/3"), (3, "S/5"), (3, "S6"), (4, "D-5"), (4, "D64"), (4, "D7"), (4, "D7/3")  …  (4, "T64"), (4, "Tm"), (5, "D"), (5, "D/3"), (5, "D/5"), (5, "D54"), (5, "T54"), (6, "D7-1"), (7, "T"), (10, "T-5")]        
 [(0, "Tm"), (2, "D"), (2, "D/3"), (2, "D/5"), (2, "D64"), (2, "D7"), (2, "D7/3"), (2, "D7/5"), (2, "D7/7"), (2, "T")  …  (4, "S65"), (4, "S65/6"), (4, "T54"), (5, "D-5"), (5, "S"), (5, "S/3"), (5, "S/5"), (5, "T64"), (7, "S6"), (7, "T-5")]      
 [(0, "Ts"), (3, "T64"), (4, "D54"), (4, "T54"), (5, "D"), (5, "D/3"), (5, "D/5"), (5, "D64"), (5, "S65"), (5, "S65/6")  …  (7, "Tm"), (8, "D-5"), (8, "D7-5"), (8, "S"), (8, "S/3"), (8, "S/5"), (9, "T-5"), (10

# # Construct the search space for note distribution to voices

In [50]:
using Combinatorics

# Finds and returns all possible distributions of the valid note classes for a given function

function getVoicingPermutations(func)
    voicing_permutations = Array{Tuple{Int64,Int64}, 1}[]
    voicings_initial = []
    
    function_map = functions[func][1]
    double_note = functions[func][3]
    bass_note = functions[func][2][1]

    active_notes = sort!([(i, x) for (i, x) in enumerate(function_map) if x > 0], by = x -> x[2])   # [position, value]
    
    filter!(e->e != bass_note, active_notes)
        
    voicing_sans_bass = copy(active_notes)
            
    if size(active_notes, 1) == 1
        push!(voicing_sans_bass, bass_note)
        push!(voicing_sans_bass, bass_note)
        push!(voicings_initial, voicing_sans_bass)
    elseif size(active_notes, 1) == 2
        for i in 1:size(double_note, 1)
            voicing_sans_bass = copy(active_notes)
            push!(voicing_sans_bass, double_note[i])
            push!(voicings_initial, voicing_sans_bass)
        end
    elseif size(active_notes, 1) == 3
        push!(voicings_initial, voicing_sans_bass)
    end
                    
    for entry in voicings_initial
        voicings = collect(permutations(entry))
        
        for j in voicings
            push!(voicing_permutations, j)
        end
    end
    
    voicing_permutations = unique(voicing_permutations)   
    for p in voicing_permutations
        pushfirst!(p, bass_note)
    end
    
    return voicing_permutations
end
        
# Generates the candidate solutions of note distribution for a given function
               
function generateCandidates(func)
    voicing_permutations = getVoicingPermutations(func)
    valid_voicings = Array{Tuple{Int64,Int64},1}[]
    note_repository = []   # [[all C's], [all C#'s], [all D's], ..., [all B's]]
    for note_class in 1:12
        push!(note_repository, [x for x in 40:80 if x%12 == (note_class-1)])
    end

    
    
    for permutation in voicing_permutations
        #println(permutation)
        voicing = []
        for b in note_repository[permutation[1][1]]
            if b in 40:60
                for t in note_repository[permutation[2][1]]
                    if t in 47:68 && t >= b && t-b <= 20
                        for a in note_repository[permutation[3][1]]
                            if a in 52:72 && a >= t && a-t <= 12                                 
                                for s in note_repository[permutation[4][1]]
                                    if s in 59:80 && s >= a && s-a <= 12  
                                        voicing = Tuple{Int64,Int64}[]
                                        push!(voicing, (b, permutation[1][2]), (t, permutation[2][2]), (a, permutation[3][2]), (s, permutation[4][2])) 
                                        push!(valid_voicings, voicing)
                                    end                                
                                end           
                            end    
                        end
                        
                    end
                end       
            end
        end
    end
    
    ## Enforce Special rules
    
    if (func == "T-5") || (func == "D-5")
        placeholder = Array{Tuple{Int64,Int64},1}[]
        for voicing in valid_voicings
            if (voicing[4][2] == 1) && (voicing[4][1] - voicing[3][1] == 9 || voicing[4][1] - voicing[3][1] == 8)
                push!(placeholder, voicing)
            end
        end
        valid_voicings = placeholder
    end
    return(valid_voicings)
end
        
# Initialises all possible permutations of every function with regards to key and scale
function createVoicingPermutations()

    function_permutations = Dict{String, Array{Any, 1}}()

    for keys in keys(functions)
        entry = unique(generateCandidates(keys))
        function_permutations[keys] = unique(generateCandidates(keys))
    end
    
    return function_permutations
end
        
function_permutations = createVoicingPermutations()       
#display(function_permutations["D"])   

Dict{String,Array{Any,1}} with 27 entries:
  "D54"  => Any[Tuple{Int64,Int64}[(48, 1), (53, 4), (55, 5), (60, 1)], Tuple{I…
  "S/5"  => Any[Tuple{Int64,Int64}[(41, 5), (58, 1), (61, 3), (65, 5)], Tuple{I…
  "T"    => Any[Tuple{Int64,Int64}[(41, 1), (56, 3), (60, 5), (65, 1)], Tuple{I…
  "Ts"   => Any[Tuple{Int64,Int64}[(49, 1), (53, 3), (56, 5), (65, 3)], Tuple{I…
  "S/3"  => Any[Tuple{Int64,Int64}[(49, 3), (58, 1), (65, 5), (65, 5)], Tuple{I…
  "T/5"  => Any[Tuple{Int64,Int64}[(48, 5), (53, 1), (56, 3), (65, 1)], Tuple{I…
  "D7/5" => Any[Tuple{Int64,Int64}[(43, 5), (60, 1), (64, 3), (70, 7)], Tuple{I…
  "Dm"   => Any[Tuple{Int64,Int64}[(51, 1), (55, 3), (58, 5), (63, 1)], Tuple{I…
  "T/3"  => Any[Tuple{Int64,Int64}[(44, 3), (53, 1), (60, 5), (65, 1)], Tuple{I…
  "D/5"  => Any[Tuple{Int64,Int64}[(43, 5), (60, 1), (64, 3), (67, 5)], Tuple{I…
  "T-5"  => Any[Tuple{Int64,Int64}[(41, 1), (53, 1), (56, 3), (65, 1)], Tuple{I…
  "D64"  => Any[Tuple{Int64,Int64}[(48, 1), (53, 4), (56, 6), (60,

In [52]:
#Global Initialisations

Tonics = ["T", "T/3", "T/5", "T-5"]
Tonics_dependant = ["T64", "T54"]
Subdominants = ["S", "S/3", "S/5", "S65", "S65/6", "S6"]
Dominants = ["D", "D/3", "D/5", "D-5", "D64", "D54", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7"]
Mediants = ["Ts", "Tm", "Ss", "Dm"]
beat_accentuation = Dict("O" => 1, "W" => 2, "S" => 3, "A" => 4)
dominant_order = Dict("D" => 3, "D/3" => 3, "D/5" => 3, "D-5" => 3, "D64" => 1, "D54" => 2, "D7" => 5, "D7-1" => 4, "D7-5" => 5, "D7/3" => 5, "D7/5" => 5, "D7/7" => 5)

if scale == "Major"
    valid_function_table = Dict(
    "T"    => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54", "Ts", "Tm", "Ss"],
    "S"    => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "T64", "T54", "D64", "D54", "Ts", "Tm", "Ss"],
    "D"    => ["T", "T-5", "T/3", "T/5", "S/3", "D", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ts"],
    "T-5"  => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54", "Ts", "Tm", "Ss"],
    "D-5"  => ["T", "T-5", "T/3", "T/5", "S/3", "D", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ts"],
    "T/3"  => ["T", "S", "S/3", "S6", "S65", "S65/6", "D", "D/3", "D/5", "D7-5", "D7/5", "D64", "D54", "Tm", "Ts"],
    "S/3"  => ["T", "T/3", "T/5", "S", "S6", "D", "D/3", "D7", "D7/3", "T64", "T54", "D64", "D54", "Ts"],
    "D/3"  => ["T", "T-5", "S/3", "D", "D7", "D7-5", "Ts"],
    "T/5"  => ["S", "S/3", "S6", "S65", "S65/6", "D"],
    "S/5"  => ["T", "S6", "S65", "S65/6"],
    "D/5"  => ["T", "T/3"],
    "D64"  => ["D", "D7-5", "D54"],
    "T64"  => ["T", "T54", "T64"],
    "T54"  => ["T", "T54"],
    "D54"  => ["D", "D54"],
    "D7"   => ["T", "T-5", "D7", "D7/3", "D7/5", "D7/7", "Ts", "T64", "T54"],
    "D7/3" => ["T", "T-5", "D7", "D7/3", "D7/5", "D7/7", "Ts"],
    "D7/5" => ["T", "D7", "D7/3", "D7/5", "D7/7"],
    "D7/7" => ["T/3", "D7", "D7/3", "D7/5", "D7/7"],
    "D7-1" => ["T", "T/3", "D7", "D7-1", "D7/3", "D7/5", "D7/7", "Ts"],
    "D7-5" => ["T", "D7-5"],
    "S65"  => ["T/5", "D", "D64", "S65"],
    "S65/6"=> ["D", "D7", "D7-1", "D7-5", "T/5", "S65/6"],
    "S6"   => ["T/3", "D", "D7", "D7-5", "D64", "S6"],
    "Ts"   => ["T/3", "S", "S/3", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D7", "D7-1", "D7/3", "Ts", "Ss", "Tm"],
    "Tm"   => ["S", "D/3", "D7/3", "Ts", "Tm"],
    "Ss"   => ["T", "T-5", "T/3", "T/5", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ss"])
    # ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "T64", "T54", "D64", "D54", "Ts", "Tm", "Ss"],
elseif scale == "Minor"
    valid_function_table = Dict(
    "T"    => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54", "Ts", "Tm", "Dm"],
    "S"    => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "T64", "T54", "D64", "D54", "Ts", "Tm", "Dm"],
    "D"    => ["T", "T-5", "T/3", "T/5", "S/3", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ts"],
    "T-5"  => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54", "Ts", "Tm", "Dm"],
    "D-5"  => ["T", "T-5", "T/3", "T/5", "S/3", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ts"],
    "T/3"  => ["T", "S", "S/3", "S6", "S65", "S65/6", "D", "D/3", "D/5", "D7-5", "D7/5", "D64", "D54", "Tm", "Ts"],
    "S/3"  => ["T", "T/3", "T/5", "S", "S6", "D", "D/3", "D7", "D7/3", "T64", "T54", "D64", "D54", "Ts"],
    "D/3"  => ["T", "T-5", "S/3", "D", "D7", "D7-5"],
    "T/5"  => ["S", "S/3", "S6", "S65", "S65/6", "D"],
    "S/5"  => ["T", "S6", "S65", "S65/6"],
    "D/5"  => ["T", "T/3"],
    "D64"  => ["D", "D7-5", "D54"],
    "T64"  => ["T", "T54", "T64"],
    "T54"  => ["T", "T54"],
    "D54"  => ["D", "D54"],
    "D7"   => ["T", "T-5", "D7", "D7/3", "D7/5", "D7/7", "T64", "T54", "Ts"],
    "D7/3" => ["T", "T-5", "D7", "D7/3", "D7/5", "D7/7"],
    "D7/5" => ["T", "D7", "D7/3", "D7/5", "D7/7"],
    "D7/7" => ["T/3", "D7", "D7/3", "D7/5", "D7/7"],
    "D7-1" => ["T", "T/3", "D7", "D7-1", "D7/3", "D7/5", "D7/7", "Ts"],
    "D7-5" => ["T", "D7-5"],
    "S65"  => ["T/5", "D", "D64", "S65"],
    "S65/6"=> ["D", "D7", "D7-1", "D7-5", "T/5", "S65/6"],
    "S6"   => ["T/3", "D", "D7", "D7-5", "D64", "S6"],
    "Ts"   => ["T/3", "S", "S/3", "S6", "S65", "S65/6", "D", "D-5", "D7", "D7-1", "Ts", "Tm"],
    "Tm"   => ["S", "D/3", "D7/3", "Ts", "Tm"],
    "Dm"   => ["S/3", "Ts", "Dm"])
    # ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "T64", "T54", "D64", "D54", "Ts", "Tm", "Dm"]
end

time_signature = (4, 2)
    
println(valid_function_table["T"])

["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54", "Ts", "Tm", "Dm"]


# # Rhythm function

In [53]:
function rhythmify(time_signature, original) # (beats per bar, subdivisions) (3,1), (3, 2), (4,1), (4, 2)
    
    rhythm_map = String[]
    
    beat_remainder = size(original, 2) % (time_signature[1] * time_signature[2])
    
    rhythms = Dict(
        (3, 1) => ["S", "W", "W"],
        (3, 2) => ["S", "O", "W", "O", "W", "O"],
        (4, 1) => ["S", "W", "S", "W"],
        (4, 2) => ["S", "O", "W", "O", "S", "O",  "W", "O"]
    )

    ## Create basic rhythm map
    if beat_remainder == time_signature[2]
        for i in 1:time_signature[2]
            push!(rhythm_map, rhythms[time_signature][end - (time_signature[2] - i)])
        end
            
        for i in 0:(size(original, 2) - 1-time_signature[2])
            push!(rhythm_map, rhythms[time_signature][(i % (time_signature[1]*time_signature[2]) + 1)])
        end
    else
        for i in 0:(size(original, 2) - 1)
            push!(rhythm_map, rhythms[time_signature][(i % (time_signature[1]*time_signature[2]) + 1)])
        end
    end
    return rhythm_map
end

display(rhythmify((4, 2), data[:, 5:end]))

12-element Array{String,1}:
 "S"
 "O"
 "W"
 "O"
 "S"
 "O"
 "W"
 "O"
 "S"
 "O"
 "W"
 "O"

In [54]:
#=
# Child : [1:(1:function1_cost, 2:"function1_name", 3:[function1_voicing_list]), 2:(1:function2_cost, 2:"function2_name", , 3:[function2_voicing_list]), ...]
function function_cost(child, rhythm_map, function_only=false, debug=false)
    
    current_child = child[end]
    current_function = current_child[2]
    current_index = size(child, 1)
    previous_child = child[end-1]
    previous_function = previous_child[2]
    previous_index = size(child, 1) - 1
    timestep = size(child, 1)
    
    if function_only
        #total_cost = sum([timestep[1] for timestep in child])
        total_cost = current_child[1]
    else
        total_cost = previous_child[4] + current_child[1]
    end

    
    # Chord repetition can only happen from stronger to weaker beats. Otherwise, cost is set by distance to original tonal_map.
    if (previous_function == current_function) && !(rhythm_map[timestep] == "A" || rhythm_map[timestep-1] == "A")
        if (beat_accentuation[rhythm_map[timestep]] >= beat_accentuation[rhythm_map[timestep-1]])
            if debug
                println("Chord repetition can only happen from stronger to weaker beats. Otherwise, cost is set by distance to original tonal_map.")
            end
            total_cost += 10000
        end
    end
    
    # /5 chords     
    # T5, S5, D5 requires same function type before and after WORKS
    if (previous_function == "T/5" || previous_function == "S/5" || previous_function == "D/5") && !function_only
        if string(child[end-2][2][1]) != string(current_function[1])
            if debug
                println("T5, S5, D5 requires same function type before and after")
            end

            total_cost += 10000
        end   
    end

        
    ## Suspended chords
        
    # Augmented chords must lie on a more accentuated beat than its resolution
    if ((previous_function == "T64") || previous_function == "T54" || (previous_function == "D64") 
                || previous_function == "D54") && (beat_accentuation[rhythm_map[previous_index]] < beat_accentuation[rhythm_map[current_index]])
        
        if debug
            println("T64, D64 must come on a stronger beat than their resolution")
        end
        total_cost += 10000
    end

    # D64 is introduced from T or S WORKS
    if current_function == "D64" && !(string(previous_function[1]) == "T" || string(previous_function[1]) == "S")
        if debug
            println("D64 is introduced from T or S")
        end
        total_cost += 10000
    end

    # D64 becomes D54 or D WORKS
    if previous_function == "D64" && !(current_function == "D54" || current_function == "D")
        if debug
            println("D64 becomes D54 or D")
        end
        total_cost += 10000
    end

    # D54 becomes D WORKS
    if previous_function == "D54" && current_function != "D"

        if debug
            println("D54 becomes D")
        end
        total_cost += 10000
    end

    # T64 is introduced from S or D7 WORKS
    if current_function == "T64" && !(occursin(r"D7", string(previous_function)) || string(previous_function[1]) == "S")

        if debug
            println("T64 is introduced from S or D7")
        end
        total_cost += 10000
    end

    # T64 becomes T54 or T WORKS
    if previous_function == "T64" && !(current_function == "T54" || current_function == "T")
        if debug
            println("T64 becomes T54 or T")
        end
        total_cost += 10000
    end

    # T54 becomes T WORKS
    if previous_function == "T54" && (current_function != "T")
        #println()
        if debug
            println("T54 becomes T")
        end
        total_cost += 10000
    end
    
    #if total_cost > 999
        
        #return total_cost
    #end

    ## Dominants
    # Dominants can only go to T, S3 
    if previous_function in Dominants && !(string(current_function[1]) == "T" || current_function == "S/3" || current_function in Dominants)
        if debug
            println("Dominants can only go to T, S3 or more complex D's")
        end
        total_cost += 10000
            
        ## D7
    
    else
        if previous_function == "D7" && !(current_function == "T-5" || current_function == "Ts")
            if debug
                println("D7 can only go to T-5, Ts")
            end
            total_cost += 10000
        end

        if previous_function == "D7-1" && !(current_function == "T" || current_function == "T/3" || current_function == "Ts" || 
                    current_function == "D7" || current_function == "D7/3" || current_function == "D7/5" || current_function == "D7/7" || current_function == "D7-1")
            if debug
                println("D7 can only go to T-5, T/3, Ts or more complex D7")
            end
            total_cost += 10000
        end

        if previous_function == "D7-5" && current_function != "T"
            if debug
                println("D7-5 can only go to T")
            end
            total_cost += 10000
        end

        if previous_function == "D7/3" && current_function != "T"
            if debug
                println("D7/3 can only go to T")
            end
            total_cost += 10000
        end

        if previous_function == "D7/5" && !(current_function == "T")
            if debug
                println("D7/5 can only go to T")
            end
            total_cost += 10000
        end

        if current_function == "D7/5" && !(previous_function == "T" || previous_function == "T/3")
            if debug
                println("D7/5 can come after T or T/3")
            end
            total_cost += 10000
        end 

        if previous_function == "D7/7" && !(current_function == "T/3")
            if debug
                println("D7/7 can only go to T/3")
            end
            total_cost += 10000
        end
    end

    # Dominants can only go to more complex D's
    if previous_function in Dominants && current_function in Dominants && dominant_order[previous_function] > dominant_order[current_function]
        if debug
            println("Dominants can only go to more complex D's")
            total_cost += 10000
        end
    end
    
    
        
            
    ## Mediants
    
    # Don't go from a submediant to its chief
    if (previous_function == "Ts" && string(current_function[1]) == "T") || (previous_function == "Tm" && string(current_function[1]) == "D") || 
        (previous_function == "Ss" && string(current_function[1]) == "S") || (previous_function == "Tm" && string(current_function[1]) == "T")
        if debug
            println("Don't go from a submediant to its chief")
        end
        total_cost += 10000
    end
    
    # Ts -> S/3 or Ss -> D7-1 must decrease in rhythmic accentuation
    if ((previous_function == "Ts" && current_function == "S/3") || (previous_function == "Ss" && current_function == "D7-1")) && 
        beat_accentuation[rhythm_map[previous_index]] < beat_accentuation[rhythm_map[current_index]]
        if debug
            println("Ts -> S/3 or Ss -> D7-1 must decrease in rhythmic accentuation")
        end
        total_cost += 10000
    end
    
    # T -> Tm -> D is not allowed
    if i > 2 && (previous_function == "Tm" || previous_function == "Ts" || previous_function == "Dm")
        ancient_child = child[end-2][1:2]
        ancient_function = previous_child[2]
        ancient_index = size(child, 1) - 2 
        
        if (string(ancient_function[1]) == "T" && previous_function == "Tm" && string(current_function[1]) == "D")
            if debug
                println("T -> Tm -> D is not allowed")
            end
            total_cost += 10000
        end 
    end
    
    # Avoid repeating /3 or /5 chords
    
    if (current_function == "T/3" && previous_function == "T/3") || (current_function == "S/3" && previous_function == "S/3") || (current_function == "D/3" && previous_function == "D/3") ||
        (current_function == "T/5" && previous_function == "T/5") || (current_function == "S/5" && previous_function == "S/5") || (current_function == "D/5" && previous_function == "D/5")
        if debug
            println("Don't repeat /3 or /5 chords")
        end
        total_cost += 10000
    end
    
    
    
    if total_cost > 999

        return total_cost
    end
    
    return total_cost
end

    
hidden_parallell = [(0, "D7-1", [(48, 1), (58, 7), (67, 5), (75, 3)], 2, 7), (0, "D7-1", [(49, 1), (56, 5), (65, 3), (77, 3)], 2, 9)]

rhythm_map = ["S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "A", "A"]
                
println(function_cost(hidden_parallell, rhythm_map, false, true))

=#

In [55]:
# Child : [1:(1:function1_cost, 2:"function1_name", 3:[function1_voicing_list]), 2:(1:function2_cost, 2:"function2_name", , 3:[function2_voicing_list]), ...]
function function_cost(child, rhythm_map, function_only=false, debug=false)
    
    current_child = child[end]
    current_function = current_child[2]
    current_index = size(child, 1)
    previous_child = child[end-1]
    previous_function = previous_child[2]
    previous_index = size(child, 1) - 1
    timestep = size(child, 1)
    
    #valid_function_table
    
    if function_only
        #total_cost = sum([timestep[1] for timestep in child])
        total_cost = current_child[1]
    else
        total_cost = previous_child[4] + current_child[1]
    end
        
    if !(current_function in valid_function_table[previous_function])
        total_cost += 10000
    end
        
    
    # Chord repetition can only happen from stronger to weaker beats. Otherwise, cost is set by distance to original tonal_map.
    if (previous_function == current_function) && !(rhythm_map[timestep] == "A" || rhythm_map[timestep-1] == "A")
        if (beat_accentuation[rhythm_map[timestep]] >= beat_accentuation[rhythm_map[timestep-1]])
            if debug
                println("Chord repetition can only happen from stronger to weaker beats. Otherwise, cost is set by distance to original tonal_map.")
            end
            total_cost += 10000
        end
    end
    
    # /5 chords     
    # T5, S5, D5 requires same function type before and after WORKS
    if (previous_function == "T/5" || previous_function == "S/5" || previous_function == "D/5") && !function_only
        if string(child[end-2][2][1]) != string(current_function[1])
            if debug
                println("T5, S5, D5 requires same function type before and after")
            end

            total_cost += 10000
        end   
    end

        
    ## Suspended chords
        
    # Augmented chords must lie on a more accentuated beat than its resolution
    if ((previous_function == "T64") || previous_function == "T54" || (previous_function == "D64") 
                || previous_function == "D54") && (beat_accentuation[rhythm_map[previous_index]] < beat_accentuation[rhythm_map[current_index]])
        
        if debug
            println("T64, D64 must come on a stronger beat than their resolution")
        end
        total_cost += 10000
    end
    
    # Ts -> S/3 or Ss -> D7-1 must decrease in rhythmic accentuation
    if ((previous_function == "Ts" && current_function == "S/3") || (previous_function == "Ss" && current_function == "D7-1")) && 
        beat_accentuation[rhythm_map[previous_index]] < beat_accentuation[rhythm_map[current_index]]
        if debug
            println("Ts -> S/3 or Ss -> D7-1 must decrease in rhythmic accentuation")
        end
        total_cost += 10000
    end
    
    if total_cost > 999

        return total_cost
    end
    
    return total_cost
end

    
hidden_parallell = [(0, "D7-1", [(48, 1), (58, 7), (67, 5), (75, 3)], 2, 7), (0, "T", [(49, 1), (56, 5), (65, 3), (77, 3)], 2, 9)]

rhythm_map = ["S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "A", "A"]
                
println(function_cost(hidden_parallell, rhythm_map, false, true))

2


In [56]:

## Function for calculating the voicing cost between neighbouring voicings
# Child : [1:(1:function1_cost, 2:"function1_name", 3:[function1_voicing_list]), 2:(1:function2_cost, 2:"function2_name", , 3:[function2_voicing_list]), ...]
function voicing_cost(child, rhythm_map_local, full=false, debug=false)
    
    if full
        total_cost = 0
        from = 2
        mediant_counter = 0
        bass_list = []
        tenor_list = []
        alto_list = []
        soprano_list = []
    else
        from = size(child, 1)
        total_cost = child[end-1][5]
    end
    
    for i in from:size(child, 1)
            
        
        current_notes = child[i][3]
        current_function = child[i][2]
        current_index = size(child, 1)
        previous_notes = child[i-1][3]
        previous_function = child[i-1][2]
                  
        ## TEST ZONE
            
        ## END TEST
            
        dist = [current_notes[1][1] - previous_notes[1][1], current_notes[2][1] - previous_notes[2][1], current_notes[3][1] - previous_notes[3][1], current_notes[4][1] - previous_notes[4][1]]

        #=
        # If full check, add distances to distance logs UNLESS OBJECT ORIENTED
        if full
            push(bass_list, dist[1])
            push(tenor_list, dist[2])
            push(alto_list, dist[3])
            push(soprano_list, dist[4])
        end
        =#
            
        if i > 2
            ancient_notes = child[i-2][3]
            ancient_function = child[i-2][2]
            previous_dist = [previous_notes[1][1] - ancient_notes[1][1], previous_notes[2][1] - ancient_notes[2][1], previous_notes[3][1] - ancient_notes[3][1], previous_notes[4][1] - ancient_notes[4][1]]
               
            # bass stuff
            if abs(previous_dist[1]) > 2 && !(sign(previous_dist[1]) != sign(dist[1])) && abs(dist[1]) > 2
                if debug
                    println("Leaps in bass should be countered by a move in the other direction")
                end
                total_cost += 2              
            elseif (sign(previous_dist[1]) != sign(dist[1])) #|| dist[1] == 0 ### siste ledd test
                if debug
                    println("Encourage walking bass")
                end
                total_cost += 1              
            end
                    
            if abs(previous_dist[4]) > 2 && abs(dist[1]) > 2
                if debug
                    println("Leaps in bass should be countered by a move in the other direction")
                end
                total_cost += 2              
            end
                
        end

        ### Voice specific rules
        
        # Avalanche check

        if abs(reduce(+, map(x -> sign(x), dist))) == 4
            if rhythm_map_local[i] == "A" && reduce(+, map(x -> sign(x), dist)) == -4 && (abs(dist[4]) < 3)
                continue
            else
                if debug
                    println("Avalanche")
                end
                total_cost += 10000
            end
        end
          
        # Immediately return if cost > 1000

        if total_cost > 999 && !debug
            return total_cost 
        end


        # voice movement cost = sum of all voices movement IF it exceeds 2 (step-wise motion has no cost)

        #total_cost += reduce(+, map(x -> abs(x) > 2 ? abs(x)-2 : 0 , distance)) # Penalize jumps depending on distance
        total_cost += reduce(+, map(x -> abs(x) > 2 ? 1 : 0 , dist)) # Flat 1 penalty for leaps

            
        # distance from original cost

        # TODO: Might mess things up

            
        # Discourage chord repetition ? 

        if (reduce(+, map(x -> abs(x), dist)) == 0)
                    
            if rhythm_map_local[i] != "A"
                if debug
                    println("Repetition")
                end
                total_cost += 1
            end
        end
               
        # MAYBE: Encourage opposite movement between bass and soprano
        #if sign(dist[1]) == sign(dist[4]) && dist[1] != 0
            #total_cost += 2
        #end
            
        # Bass should be encouraged to move step-wise
        if abs(dist[1]) > 2
            total_cost += 2         
        end
                    
        # Mid voices must move < 8
        if !(dist[2] < 8) || !(dist[3] < 8)
            if debug
                println("Mid voices must move < 8")
            end
            total_cost += 10000
        end
        

        # Outer voices should only move < 8
        if !(abs(dist[1]) < 8) || !(abs(dist[4]) < 8)
            total_cost += 1
        end

            
        # Outer voices may move < 9 or 12 at a cost
        if (abs(dist[1]) >= 8 && abs(dist[1]) != 12) || (abs(dist[1]) > 7 && abs(dist[1]) < 10 && previous_function != current_function) || (abs(dist[4]) >= 10 && abs(dist[4]) != 12)
            if debug
                println("Outer voices may move < 9 or 12 at a cost")
            end
            total_cost += 10000
        end

        # Parallell primes, fifths and octaves are illegal WORKS
        if current_notes != previous_notes
            pairs = [(1, 2), (2, 3), (3, 4), (1, 3), (2, 4), (1, 4)]
            for (voice1, voice2) in pairs
                if abs(current_notes[voice1][1] - current_notes[voice2][1]) % 12== 0 && abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 0
                    if debug
                        println("Parallell prime")
                    end
                    total_cost += 10000
                elseif abs(current_notes[voice1][1] - current_notes[voice2][1]) % 12 == 7 && abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 7
                    total_cost += 10000
                    if debug
                        println("Parallell fifth")
                    end
                elseif abs(current_notes[voice1][1] - current_notes[voice2][1]) % 12 == 12 && abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 12
                    total_cost += 10000
                    if debug
                        println("Parallell octave")
                    end
                # Parallell movement from diminished fifth to fifth is illegal
                elseif abs(current_notes[voice1][1] - current_notes[voice2][1]) % 12 == 7 && 
                        abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 6
                    total_cost += 10000
                    if debug
                        println("Parallell tritone to fifth")
                    end                          
                end
            end
        end
        
        # Hidden parallell fifths and octaves are illegal WORKS!
        if (sign(dist[1]) == sign(dist[4])) && abs(dist[4]) > 2 && ((abs(current_notes[1][1] - current_notes[4][1]) % 12 == 0) || (abs(current_notes[1][1] - current_notes[4][1]) % 12 == 7))
            if debug
                println("Hidden parallell fifths and octaves are illegal")
            end
            total_cost += 10000
        end
                                
        # Immediately return if cost > 1000

        if total_cost > 999 && !debug
            return total_cost
        end

                                
        ## TEST ZONE
        
        #=
        if (previous_function == "D7-1" && current_function == "T")  
            println(previous_notes)
            println(current_notes)
            println(" ")                                 
        end 
        =#
   
        ## END TEST
              
        
        ### Chord specific rules
        ## Dominants               
        
        # From S to D, bass moves stepwise up while rest move shortest path down
        if (previous_function == "S" && current_function == "D") && dist[1] != 2 && rhythm_map_local[i] != "A"
            if debug
                    println("From S to D, bass moves stepwise up while rest move shortest path down")
                end
            total_cost += 10000
        end
                        
        if total_cost > 999 && !debug
            return total_cost 
        end
                        
        # Leading note in D goes to root, except can leap down a 3rd to 5 in mid voices # WORKS ?           
        if string(previous_function[1]) == "D" && (string(current_function[1]) == "T" && !(current_function == "Ts") && !(current_function == "Tm"))
               
            for (j, voice) in enumerate(previous_notes)
                                
                if rhythm_map_local[i] == "A" 
                    if voice[2] == 3 && !((current_notes[j][2] == 1 && dist[j] == 1) || 
                    (((current_notes[j][2] == 5 && dist[j] == -4)) && (j == 2 || j == 3) && rhythm_map[current_index] == "A"))
                        if debug
                            println("The leading note must move to 1 (or down to 5 in cadence) in T")
                        end

                        total_cost += 10000
                    end
                elseif voice[2] == 3 && current_notes[j][2] != 1 && dist[j] != 1
                    if debug
                        println("D->T: D:3 must resolve into T:1")
                    end
                    total_cost += 10000
                end
            end
                            
        end
                        
        ## D7
                                
        if (previous_function == "D7" || previous_function == "D7/3" ||  previous_function == "D7/5" || 
                                    previous_function == "D7/7" || previous_function == "D7-5") && !(current_function == "Ts")
                                    
            for (j, voice) in enumerate(previous_notes)
                
                if voice[2] == 3 && !((current_notes[j][2] == 1 && (dist[j] == 1 || dist[j] == 2)) || 
                    (((current_notes[j][2] == 5 && dist[j] == -4)) && (j == 2 || j == 3) && rhythm_map[current_index] == "A"))
                    #Voice 2 (59, 3) - (60, 1), distance 1
                    if debug
                        println("D7: The leading note must resolve into 1 (or 5 if cadence) in T") # tested
                    end
                    total_cost += 10000
                end
                if voice[2] == 7 && !(current_notes[j][2] == 3 && (dist[j] == -2 || dist[j] == -1))
                    if debug
                        #println((current_notes[j][2]))
                        #println(voice, " - ", current_notes[j], ", distance ", dist[j])
                        println("D7: The 7 must resolve into 3 in T") # tested
                    end
                    total_cost += 10000
                end
                if voice[2] == 1 && !((current_notes[j][2] == 1) || (current_notes[j][2] == 5))
                    if debug
                        println("D7: The 1 must go to 1 or 5 in T") # tested
                    end
                    total_cost += 10000
                end
                
            end                
        end
                                  
        # D7/5: bassen må føres trinnvis inn og ut av akkorden, kvarten må føres med tonegjentakelse inn og ut
        if previous_function == "D7/5"
            
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 1 && !(dist[j] == 0)
                    if debug
                        println("D7/5: The root must remain stationary") # tested
                        
                    end
                    total_cost += 10000
                end                       
            end  
                                            
            if abs(dist[1]) > 2 
                                            
                if debug
                    println("D7/5: The bass must move step-wise throughout the chord") # tested
                end
                total_cost += 10000
            end                
        end
                                        
        if current_function == "D7/5"

            for (j, voice) in enumerate(current_notes)
                if voice[2] == 1 && !(dist[j] == 0)
                    if debug
                        println("D7/5: The root must remain stationary") # Tested
                        
                    end
                    total_cost += 10000
                end                       
            end  
                                            
            if abs(dist[1]) > 2 
                                            
                if debug
                    println("D7/5: The bass must move step-wise throughout the chord") # Tested
                end
                total_cost += 10000
            end                                
                                            
        end
                                
        #D7-1
        
        if previous_function == "D7-1" && !(current_function == "Ts" || current_function == "D7" || current_function == "D7/3" || current_function == "D7/5" || 
                                            current_function == "D7/7" || current_function == "D7-1")
            sevencounter = 0
            sevenvoice = []
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 3 && !((current_notes[j][2] == 1 && (dist[j] == 1 || dist[j] == 2)) || 
                    (((current_notes[j][2] == 5 && dist[j] == -4)) && (j == 2 || j == 3) && rhythm_map[current_index] == "A"))
                    #Voice 2 (59, 3) - (60, 1), distance 1
                    if debug
                        println("D7-1: The leading note must resolve into 1 (or 5 if cadence) in T") # tested
                    end
                    total_cost += 10000
                end
                # The 7 in D7 must either go step-wise down or up
                if voice[2] == 7
                    sevencounter += 1
                    push!(sevenvoice, j)
                    if !(current_notes[j][2] == 3 && (dist[j] == -2 || dist[j] == -1) || 
                                    current_notes[j][2] == 5 && dist[j] == 2)
                        if debug
                            #println((current_notes[j][2]))
                            println("D7-1: The 7 must resolve step-wise into 3 or 5 in T") # tested
                        end
                        total_cost += 10000
                    end
                end                
            end

            if sevencounter == 2 && (current_notes[sevenvoice[1]][2] == current_notes[sevenvoice[2]][2])
                if debug
                    println("D7-1: doubled septims must move in opposite directions")
                end
                total_cost += 10000
            end
        end
                            
        ## /5 chords WORKS
                        
        # Bass and voice with /5 root must move max step-wise in and out.
        if current_function == "T/5" || current_function == "D/5" || current_function == "S/5" 

            if abs(dist[1]) > 2 
                total_cost += 10000   
            end
                            
            for (j, voice) in enumerate(current_notes)
                if voice[2] == 1 && abs(dist[j]) > 2
                    total_cost += 10000
                end
            end
        end
                        
        if previous_function == "T/5" || previous_function == "D/5" || previous_function == "S/5"
            if abs(dist[1]) > 2 
                total_cost += 10000  
            end
                            
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 1 && abs(dist[j]) > 2
                    total_cost += 10000
                end
            end
        end
        
        ## Augmented chords
                        
        # The 4 in 64 chords must be introduced stationary and resolved step-wise                
        if current_function == "T64" || current_function == "D64"
                            
            for (j, voice) in enumerate(current_notes)
                if voice[2] == 4 && dist[j] != 0
                    if debug
                        println("4 is not stationary in augmented")
                    end
                    total_cost += 10000
                end
                if voice[2] == 6 && abs(dist[j]) > 2
                    if debug
                        println("6 leaps in augmented")
                    end
                    total_cost += 2
                end
            end
        end
                        
        if previous_function == "T64" || previous_function == "D64"
            if dist[1] != 0
                total_cost += 10000 
            end
                            
            if string(current_function[1]) == string(ancient_function[1])
                total_cost += 10000
            end
                                
                            
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 4 && (dist[j] < -2 || dist[j] > 0)
                    if debug
                        println("4 is not resolved step-wise down")
                    end
                    total_cost += 10000
                end
                if voice[2] == 6 && (dist[j] < -2 || dist[j] > 0)
                    if debug
                        println("6 is not resolved step-wise down")
                    end
                    total_cost += 10000
                end
                if (voice[2] == 1 && dist[j] != 0) 
                    if debug
                        println("Root notes must be stationary")
                    end
                    total_cost += 10000                      
                end
            end
        end
        
        # The 4 in T54 must be introduced stationary and resolved step-wise down
        if current_function == "T54" || current_function == "D54"
                            
            for (j, voice) in enumerate(current_notes)
                if voice[2] == 4 && dist[j] != 0
                    if debug
                        println("4 is not stationary in augmented")
                    end
                    total_cost += 10000
                end
            end
        end
         
        if previous_function == "T54" || previous_function == "D54"
            if dist[1] != 0
                if debug
                    println("T54/D54: Bass must be stationary")
                end
                total_cost += 10000  
            end
                            
            if string(current_function[1]) == string(ancient_function[1])
                if debug
                    println("T54/D54: Not same function before and after")
                end
                total_cost += 10000
            end
                            
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 4 && (dist[j] < -2 || dist[j] > 0)
                    if debug
                        println("4 is not resolved step-wise down")
                    end
                    total_cost += 10000
                end
                if (voice[2] == 1 && dist[j] != 0) || (voice[2] == 5 && dist[j] != 0)
                    if debug
                        println("Root and fifth notes must be stationary")
                    end
                    total_cost += 10000                      
                end
            end
            
        end
                                
                            
        ## /3 chords
                                
        if (previous_function == "T/3" && current_function == "D/3") || (previous_function == "S/3" && current_function == "T/3")                    
            if sign(dist[1]) != sign(-1)                              
                if debug
                    println("T/3 -> D/3 or S/3 -> T/3 needs to have bass leaping downwards")
                end
                total_cost += 10000
            end
        end
                                
        if (previous_function == "T/3" && current_function == "S/3")                  
            if sign(dist[1]) != sign(1)                              
                if debug
                    println("T/3 -> S/3 needs to have bass leaping upwards")
                end
                total_cost += 10000
            end
        end
                                    
        ## Mediants
                                    
        # The 3 of D becomes 3 of Ts
                               
        if ((previous_function == "D" || previous_function == "D7"  || previous_function == "D7-1")  && current_function == "Ts")  
                                        
           for (j, voice) in enumerate(previous_notes)
                if voice[2] == 3 && current_notes[j][2] != 3
                    if debug
                        println("D->Ts: 3 in D must resolve into 3 in Ts")
                    end
                    total_cost += 10000
                end
            end                                     
        end
                               
        # Mediant Counter
        if full && (current_function == "Ts" || current_function == "Ss" || current_function == "Tm" || current_function == "Dm")
            mediant_counter += 1
        end
        
        # No more than two mediants per 16 timesteps
        if full
            if mediant_counter > 2
                if debug
                    println("Too many mediants in the final sequence.")
                end
                total_cost += 1000
            end
        end
         
        # /3 chords together are encouraged, but only if correct voicing is used
        if i > 2 && (previous_function == "T/3" || previous_function == "S/3" || previous_function == "D/3")

            if (ancient_function == "T/3" && previous_function == "D/3") || 
                (ancient_function == "S/3" && previous_function == "T/3") || 
                (ancient_function == "T/3" && previous_function == "S/3")
                total_cost -= 1
                                
                # Leaps in bass should be countered by a move in the other direction
                if ((abs(dist[1]) > 2 && abs(previous_dist[1]) > 2))
                    if debug
                        println("Two /3 chords with a leap in bass need to be countered by a stepwise motion")
                    end
                    total_cost += 10000           
                end
                if  !(sign(previous_dist[1]) != sign(dist[1]))
                    if debug
                        println("Two /3 chords with a leap must be resolved with motion in the other direction")
                    end
                    total_cost += 10000
                end
                if (sign(previous_dist[1]) == sign(previous_dist[4]))
                    if debug
                        println("Only use two /3 chords in motion when soprano is moving opposite to the bass or is flat")
                    end
                    total_cost += 10000
                end
                                
            elseif (ancient_function == "T" && previous_function == "D/3" && current_function == "T") || (ancient_function == "S" && previous_function == "T/3" && current_function == "S") ||
                                    (ancient_function == "D" && previous_function == "T/3" && current_function == "D") || 
                                    (ancient_function == "T" && previous_function == "S/3" && current_function == "T") ||
                                    (ancient_function == "D" && previous_function == "T/3" && current_function == "S") || 
                                    (ancient_function == "S" && previous_function == "T/3" && current_function == "D")
                                    
                total_cost -= 1
                                    
            elseif (ancient_function == "T" && previous_function == "T/3" && current_function == "S") || (ancient_function == "T" && previous_function == "T/3" && current_function == "D") ||
                                    (ancient_function == "S" && previous_function == "S/3" && current_function == "T") || (ancient_function == "D" && previous_function == "D/3" && current_function == "T-5") ||
                                    (ancient_function == "D" && previous_function == "D/3" && current_function == "T") || (ancient_function == "T" && previous_function == "D/3" && current_function == "D") ||
                                    (ancient_function == "T" && previous_function == "S/3" && current_function == "S") || (ancient_function == "S" && previous_function == "T/3" && current_function == "T")
                    
                total_cost -= 1                     
            end
        end
                          
                                    
        # TODO
    
    end                                 
    
    return total_cost
        
end
            
#println(voicing_cost((0, "T", [()])), ["S", "W"])

#println(sign(48 - 60) == sign(72 - 77) && (abs(-5) > 2) && ((abs(48 - 72) % 12 == 0) || (abs(48 - 72) % 12 == 7)))
                
 #previous [(60, 1), (60, 1), (67, 5), (77, 4)]
 #current [(48, 1), (64, 3), (67, 5), (72, 1)]
                
                
hidden_parallell = [(0, "D7-1", [(55, 5), (58, 7), (58, 7), (63, 3)], 2, 7), (0, "T", [(53, 1), (56, 3), (60, 5), (65, 1)], 2, 9)]
                                    
Tuple{Int64,Int64}[(55, 5), (58, 7), (58, 7), (63, 3)]
Tuple{Int64,Int64}[(53, 1), (56, 3), (60, 5), (65, 1)]
rhythm_map = ["S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "A", "A"]
                
println(voicing_cost(hidden_parallell, rhythm_map, false, true))
                
#dist = [0, 0, 0, 0]
#println((reduce(+, map(x -> abs(x), dist)) == 0) && "S" != "T")
                



7


In [57]:
# Rate the voicing permutation with regards to the original score
function distance_cost(child, timestep, first=false, debug=false)
    
    if first
        total_dist = 0
    else
        total_dist = child[end-1][6]
    end
        
    local_dist = 0
    voices = child[end][3]
    active_notes = getActiveNotes(timestep)
    
    if debug
        println("Active notes: ", active_notes)
        println("Voicing: ", voices)
    end
    
    for (i, voice) in enumerate(voices)
        
        if i == 1
            local_dist += abs(active_notes[1] - voice[1])
            if debug
                println("Bass distance: ", abs(active_notes[1] - voice[1]))
            end
        #=
        elseif i == 2 || i == 3
            if debug
                println("Mid-voice distance: ", minimum([abs(active_note - voice[1]) for active_note in active_notes]))
            end
            total_dist += minimum([abs(active_note - voice[1]) for active_note in active_notes])
        =#
        elseif i == 4
            local_dist += abs(active_notes[end] - voice[1])
            if debug
                println("Soprano distance: ", abs(active_notes[end] - voice[1]))
            end
        end
    end
    
    # To normalize voicing costs, divide by 12 and round to nearest integer
        
    local_dist = floor(Int, local_dist / 12)
        
    total_dist += local_dist
            
    return total_dist
 
end

#=
#println(distance_cost([(5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), (0, "S6", [(54, 1), (65, 1), (69, 3), (90, 6)], 10, 4)], 1, true, true))
# child, solution_list[1], rhythm_map_local, min_list, false
function function_voice_heuristic(child, best_solution, rhythm_map, min_list, function_potential_local, debug=true)
    
    current_depth = size(child, 1)
    total_depth = size(best_solution[2], 1)
    best_total_score = best_solution[1]
    best_solution = best_solution[2]
    println(best_total_score)
            
    # Try with average voice cost of the best solution up to this point
            
    best_average_voice_cost = reduce(+, [voice_cost[5] for voice_cost in best_solution[1:current_depth]]) / 2
    println(best_average_voice_cost) 
            
            
end
        

        
child = [(10, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), 
            (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4), 
            (2, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10), 
            (4, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16), 
            (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24), 
            (6, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27), 
            (4, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37), 
            (8, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38), 
            (4, "S/3", [(45, 3), (53, 1), (65, 1), (72, 5)], 38, 40), 
            (2, "Tm", [(52, 1), (55, 3), (67, 3), (71, 5)], 40, 43), 
            (6, "S", [(53, 1), (57, 3), (60, 5), (65, 1)], 46, 45), 
            (10, "D7-1", [(50, 5), (53, 7), (65, 7), (71, 3)], 56, 54)] # <-

best = (146.0, [(10, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), 
        (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4), 
        (2, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10), 
        (4, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16), 
        (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24), 
        (6, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27), 
        (4, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37), 
        (8, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38), 
        (4, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42), 
        (2, "D/3", [(47, 3), (50, 5), (55, 1), (67, 1)], 40, 44), 
        (8, "T", [(48, 1), (52, 3), (60, 1), (67, 5)], 48, 45), 
        (6, "S", [(53, 1), (57, 3), (60, 5), (65, 1)], 54, 49), # <-
        (18, "D-5", [(55, 1), (55, 1), (59, 3), (67, 1)], 72, 49), 
        (6, "D7", [(55, 1), (59, 3), (62, 5), (65, 7)], 78, 52), 
        (10, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 88, 53), 
        (6, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 94, 52)]) 
        
rhythm_map = ["S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "A", "A"]
        
min_list = [10, 0, 2, 4, 0, 2, 4, 0, 4, 2, 4, 4, 4, 6, 10, 6]
        
function_voice_heuristic(child, best, rhythm_map, min_list, true)
        
#1 [(10, "T")]                                                                                                                                                                                                                                                
#2 [(0, "S6"), (0, "Ss"), (4, "D7-1"), (4, "S"), (4, "S/3"), (4, "S/5"), (6, "D"), (6, "D/3"), (6, "D/5"), (6, "D7")  …  (6, "D7/7"), (6, "Ts"), (8, "D-5"), (8, "T"), (8, "T-5"), (8, "T/3"), (8, "T/5"), (8, "Tm"), (16, "S65"), (16, "S65/6")]             
#3 [(2, "T"), (2, "T-5"), (2, "T/3"), (2, "T/5"), (2, "Ts"), (4, "S"), (4, "S/3"), (4, "S/5"), (4, "T64"), (4, "Tm")  …  (8, "S65/6"), (8, "Ss"), (12, "D7-1"), (16, "D"), (16, "D/3"), (16, "D/5"), (18, "D7"), (18, "D7/3"), (18, "D7/5"), (18, "D7/7")]    
#4 [(4, "S"), (4, "S/3"), (4, "S/5"), (6, "D7-5"), (6, "S6"), (6, "Ss"), (6, "Ts"), (8, "D-5"), (8, "D7-1"), (8, "T-5")  …  (10, "T"), (10, "T/3"), (10, "T/5"), (12, "D"), (12, "D/3"), (12, "D/5"), (12, "D7"), (12, "D7/3"), (12, "D7/5"), (12, "D7/7")]   
#5 [(0, "T54"), (2, "D64"), (2, "D7-5"), (2, "T"), (2, "T/3"), (4, "D-5"), (4, "D7"), (4, "D7/3"), (4, "D7/7"), (4, "S")  …  (4, "Tm"), (6, "D54"), (6, "Ts"), (8, "S6"), (8, "Ss"), (10, "S65"), (10, "S65/6"), (12, "D7-1"), (16, "D"), (16, "D/3")]        
#6 [(2, "Ts"), (4, "S"), (4, "S/3"), (4, "S/5"), (4, "T-5"), (6, "D54"), (6, "T"), (6, "T/3"), (6, "T/5"), (8, "D")  …  (10, "D7-5"), (10, "S6"), (10, "S65"), (10, "S65/6"), (10, "Ss"), (12, "D7-1"), (16, "D7"), (16, "D7/3"), (16, "D7/5"), (16, "D7/7")] 
#7 [(4, "D64"), (4, "T"), (4, "T/3"), (4, "T/5"), (4, "Ts"), (6, "S"), (6, "S/3"), (6, "S/5"), (6, "T-5"), (6, "T64")  …  (18, "D"), (18, "D/3"), (18, "D/5"), (22, "D7"), (22, "D7-1"), (22, "D7/3"), (22, "D7/5"), (22, "D7/7"), (26, "S6"), (26, "Ss")]    
#8 [(0, "S"), (0, "S/3"), (0, "S/5"), (0, "T64"), (2, "T"), (2, "T/3"), (2, "T/5"), (2, "Ts"), (4, "D7"), (4, "D7-1")  …  (4, "Ss"), (4, "T-5"), (6, "S65"), (6, "S65/6"), (8, "D"), (8, "D-5"), (8, "D/3"), (8, "D/5"), (10, "D7-5"), (24, "Tm")]            
#9 [(4, "D"), (4, "D-5"), (4, "D/3"), (4, "S"), (4, "S/3"), (4, "T54"), (4, "T64"), (6, "D64"), (6, "T"), (6, "T/3")  …  (8, "D7/3"), (8, "D7/7"), (8, "S6"), (8, "Ss"), (8, "T-5"), (10, "D54"), (10, "S65"), (10, "S65/6"), (14, "D7-5"), (16, "Tm")]       
#10 [(2, "D"), (2, "D-5"), (2, "D/3"), (2, "D/5"), (2, "Tm"), (4, "D64"), (4, "D7-5"), (4, "T"), (4, "T/3"), (4, "T/5")  …  (10, "S/3"), (10, "S/5"), (10, "S6"), (10, "Ss"), (20, "S65"), (20, "S65/6"), (22, "D7"), (22, "D7/3"), (22, "D7/5"), (22, "D7/7")]
#11 [(4, "Ts"), (6, "D"), (6, "D/3"), (6, "D/5"), (6, "S"), (6, "S/3"), (6, "S/5"), (6, "T-5"), (8, "D-5"), (8, "S65")  …  (8, "T/5"), (10, "D7-1"), (10, "Tm"), (12, "D7-5"), (14, "S6"), (14, "Ss"), (18, "D7"), (18, "D7/3"), (18, "D7/5"), (18, "D7/7")]   
#12 [(4, "D-5"), (6, "D"), (6, "D/3"), (6, "D/5"), (6, "S"), (6, "S/3"), (6, "S/5"), (8, "D54"), (8, "T-5"), (8, "Ts")  …  (12, "S65"), (12, "S65/6"), (12, "T"), (12, "T/3"), (12, "T/5"), (14, "D7"), (14, "D7/3"), (14, "D7/5"), (14, "D7/7"), (14, "Tm")]  
#13 [(4, "D7-5"), (12, "D64"), (16, "D54"), (18, "D-5"), (18, "D7"), (18, "D7-1"), (18, "D7/3"), (18, "D7/7"), (22, "D"), (22, "D/3")]                                                                                                                         
#14 [(6, "D7"), (14, "D")]                                                                                                                                                                                                                                     
#15 [(10, "T")]                                                                                                                                                                                                                                                
#16 [(6, "T")]
        
=#

distance_cost (generic function with 3 methods)

In [58]:
function getChordIndex(chord)
    if scale == "Major"
        function_index = ["S65/6", "S/5", "T", "Ts", "S/3", "T/5", "Ss", "D7/5", "T/3", "D/5", "T-5", "D64", "D/3", "D-5",
            "D7", "T64", "D7-5", "D", "Tm", "S65", "T54", "S", "D7/3", "D7-1", "S6", "D7/7", "D54"]
    elseif scale == "Minor"
        function_index = ["S65/6", "S/5", "T", "Ts", "S/3", "T/5", "Dm", "D7/5", "T/3", "D/5", "T-5", "D64", "D/3", "D-5",
            "D7", "T64", "D7-5", "D", "Tm", "S65", "T54", "S", "D7/3", "D7-1", "S6", "D7/7", "D54"]
    end
    return findall(x->x == chord, function_index)[1]
end

function getChordName(index)
    function_index = ["S65/6", "S/5", "T", "Ts", "S/3", "T/5", "Ss", "D7/5", "T/3", "D/5", "T-5", "D64", "D/3", "D-5",
            "D7", "T64", "D7-5", "D", "Tm", "S65", "T54", "S", "D7/3", "D7-1", "S6", "D7/7", "D54", "Dm"]
    return function_index[index]
end

function buildMinList(function_potential_list, function_potential_local, rhythm_map, starting_chord_index, final_chord_index, debug=false)
    
    #=
    timestep1 = Tuple{Int64,String}[(6, "T")]
    timestep2 = Tuple{Int64,String}[(3, "D7-1"), (5, "D"), (6, "S"), (7, "T")]
    timestep3 = Tuple{Int64,String}[(3, "S"), (5, "D7-1"), (5, "T"), (7, "D")]
    timestep4 = Tuple{Int64,String}[(0, "S"), (5, "T"), (7, "D7-1"), (9, "D")]
    timestep5 = Tuple{Int64,String}[(2, "D7-1"), (2, "T"), (4, "D"), (6, "S")]
    timestep6 = Tuple{Int64,String}[(6, "D"), (8, "D7-1")]
    timestep7 = Tuple{Int64,String}[(7, "T")]
    
    temppot = [timestep1, timestep2, timestep3, timestep4, timestep5, timestep6, timestep7]
    =#
    
    function_potential_reversed = reverse(function_potential_local)
    temppot = deepcopy(function_potential_list)
    
    if debug
        display(temppot)
    end
    
    minList = []
    
    
    # [1000, ((3, "D7"), (3, "S"))] [1000, ((7, "T"), (5, "D7"))] [1000, ("current", "previous")] [1000, ("current", "previous")]; 
    # [1000, ((5, "D"), (3, "S"))] [1000, ((7, "T"), (5, "T"))] [1000, ("current", "previous")] [1000, ("current", "previous")]; 
    # [1000, ((6, "S"), (3, "S"))] [1000, ((7, "T"), (7, "D"))] [1000, ("current", "previous")] [1000, ("current", "previous")]; 
    # [1000, ((7, "T"), (3, "S"))] [1000, ("current", "previous")] [1000, ("current", "previous")] [1000, ("current", "previous")]]   
    for i in 1:size(temppot, 1)
        if i == 1
            timesteparray = fill([1000, (temppot[i][starting_chord_index])], (1, 1))
            pushfirst!(minList, timesteparray)
        else
            timesteparray = fill([1000, ("current", "previous")], (size(temppot[i-1], 1), size(temppot[i], 1)))
            for (j, previous_chord) in enumerate(temppot[i-1])
                for (k, current_chord) in enumerate(temppot[i])
                    chordcombo = (previous_chord, current_chord)
                    timesteparray[((j-1)*size(temppot[i], 1))+k] = [1000, chordcombo]
                end
            end
            pushfirst!(minList, timesteparray)      
        end

    end
        
        
    rhythm_map_rev = reverse(rhythm_map)
        
        
    # Each timestep in minList contains a matrix which describes functions[i] * functions[i-1] from function_potential
        
    for (k, timestep) in enumerate(minList)
            
        shortest_found = 10000    
        # 1:[1:1000, 2:(1:(1:6, 2:"D"), 2:(1:7, 2:"T"))] 2:[1:1000, 2:(1:(1:8, 2:"D7"), 2:(1:7, 2:"T"))]]
        for (i, chordcombination) in enumerate(timestep)
                
            # function_cost(child, rhythm_map, function_only=false, debug=false) 
            if k == 1 && string(chordcombination[2][2][2]) == string(getChordName(starting_chord_index))
                chordcombination[1] = function_cost([chordcombination[2][1], chordcombination[2][2]], rhythm_map_rev, true, false)
            elseif k == 1
                #println(chordcombination[2][2])
                chordcombination[1] = 10000
            elseif k == size(minList, 1)
                previous_timestep = minList[k-1]            
                previous_matches = [prev_combo[1] for prev_combo in previous_timestep if prev_combo[2][1] == chordcombination[2]]
                shortest_path = minimum(previous_matches)
                chordcombination[1] = shortest_path + chordcombination[2][1]
            else
                if chordcombination[2][2] in function_potential_reversed[k]
                    previous_timestep = minList[k-1]
                    previous_matches = [prev_combo[1] for prev_combo in previous_timestep if prev_combo[2][1] == chordcombination[2][2]]
                    shortest_path = minimum(previous_matches)
                    chordcombination[1] = function_cost([chordcombination[2][1], chordcombination[2][2]], rhythm_map_rev, true, false) + shortest_path
                else
                    chordcombination[1] += 1000
                end
            end
        end
    end  
            
    # minList[i][:, j] for all combinations with j as precursor
    minList = reverse(minList)
    if debug
        display(minList)
    end
    return minList
end

function trans(function_potential_local, rhythm_map_local, data)
    
    start = ["T"]
    leading_end = ["D", "D7-5"]
    final = ["T"]
    start_index = findfirst(x -> x == "S", rhythm_map_local)
    final_index = findlast(x -> x == "S" || x == "W", rhythm_map_local)
    leading_index = findlast(x -> x == "W" || x == "S", rhythm_map_local[1:final_index-1])

    function_potential_local = deepcopy(function_potential_local)
    

    filter!(x -> x[2] in start, function_potential_local[start_index])
    for i in leading_index:final_index-1
        filter!(x -> x[2] in Dominants, function_potential_local[i])
    #rhythm_map_local[i] = "D"
    end

    
    filter!(x -> x[2] in leading_end, function_potential_local[final_index-1])
    #rhythm_map_local[final_index-1] = "D"
    for i in final_index:size(data, 2)
        filter!(x -> x[2] in final, function_potential_local[i])
        rhythm_map_local[i] = "A" # Authentic cadence
    end
    
    #println("Rhythm map: ", rhythm_map_local)


    # Prune function_potential by beat restrictions for each chord
    for (i, potential) in enumerate(function_potential_local)
        for (j, (cost, chord)) in enumerate(potential)
            if !(rhythm_map_local[i] in functions[chord][4])
                filter!(x -> x[2] != chord, potential)
            end
        end
    end 
    ##### END BB   

    return rhythm_map_local, function_potential_local
end
                    
rhythm_map_local = ["S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O"]

#### FROM BB
function_pot = deepcopy(function_potential[1:16])
rm, fp = trans(function_pot, rhythm_map_local, data[:, 1:16])
#display(rm)
#display(fp)
                    
minList = buildMinList(function_potential_unsorted[1:16], fp, rm, 3, 3, false)

# MinList indexing = minList[timestep][currentchord, previouschord] returns the shortest possible path to end if current chord is chosen.
# NB: the cost of the current chord is NOT included, as this is covered by the incremental function cost calculation, not the heuristic.

16-element Array{Any,1}:
 Array{Any,1}[[42, (0, "T")]]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
 Array{Any,1}[[2000, ((2, "S65/6"), (4, "S65/6"))] [2000, ((3, "S/5"), (4, "S65/6"))] … [2000, ((1, "D7/7"), (4, "S65/6"))] [2000, ((1, "D54"), (4, "S65/6"))]; [10045, ((2, "S65/6"), (3, "S/5"))] [10045, ((3, "S/5"), (3, "S/5"))] … [10045, ((1, "D7/7"), (3, "S/5"))] [10045, ((1, "D54"), (3, "S/5"))]; … ; [10046, ((2, "S65/6"), (4, "D7/7"))] [10046, ((3, "S/5"), (4, "D7/7

In [59]:
function showShortestPath(minList, t)
                    
    shortest_path = []
    
    minListlocal = reverse(minList)
    
    push!(shortest_path, minListlocal[1][3, 18])
    
    #println(shortest_path)
    
    for (i, timestep) in enumerate(minListlocal[2:17-t])
        possible_solutions = minimum([(chord[1], chord) for chord in timestep if chord[2][2][2] == shortest_path[end][2][1][2]])
        push!(shortest_path, possible_solutions[2])
    end
    
    return shortest_path
     
end
## CONTINUE LATER!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
## Check consistency between heuristic and prediction
#display(showShortestPath(minList, 2))

show(minList[13][24,:])
#minList[timestep][currentchord, previouschord]

#1  (6, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 6, 0),        MinList = 
#2  (3, "D7", [(55, 1), (62, 5), (65, 7), (71, 3)], 9, 4),       MinList = 
#3  (8, "T-5", [(60, 1), (60, 1), (64, 3), (72, 1)], 17, 10),    MinList = 
#4  (0, "S", [(53, 1), (60, 5), (65, 1), (69, 3)], 17, 15),      MinList = 
#5  (2, "D7/3", [(47, 3), (65, 7), (67, 1), (74, 5)], 19, 24),   MinList = 
#6  (6, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 25, 25),      MinList = 
#7  (3, "D", [(55, 1), (62, 5), (67, 1), (71, 3)], 28, 28),      MinList = 
#8  (2, "S/3", [(57, 3), (60, 5), (65, 1), (77, 1)], 30, 29),    MinList = 
#9  (4, "Ts", [(57, 1), (57, 1), (64, 5), (72, 3)], 34, 32),     MinList = 
#10 (2, "D/3", [(59, 3), (62, 5), (67, 1), (67, 1)], 36, 36),    MinList = 
#11 (3, "D", [(55, 1), (67, 1), (71, 3), (74, 5)], 39, 45),      MinList = 
#12 (2, "T/5", [(55, 5), (64, 3), (72, 1), (79, 5)], 41, 48),    MinList = 
#13 (5, "D", [(55, 1), (67, 1), (71, 3), (74, 5)], 46, 50),      MinList = ((2, "T/5"), (5, "D")) = 23 - 5 = 18
#14 (7, "D", [(55, 1), (67, 1), (71, 3), (74, 5)], 53, 51),      MinList = ((5, "D"), (7, "D")) = 18 - 7 = 11
#15 (8, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 61, 52),      MinList = ((7, "D"), (8, "T")) = 11 - 8 = 3
#16 (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 64, 53)]      MinList = (8, T) - (3, T) = 3 - 3 = 0

# 1:S65/6, 2:S/5, 3:T, 4:Ts, 5:S/3, 6:T/5, 7:Ss
# 8:D7/5, 9:T/3, 10:D/5, 11:T-5, 12:D64, 13:D/3, 14:D-5
# 15:D7, 16:T64, 17:D7-5, 18:D, 19:Tm, 20:S65, 21:T54
# 22:S, 23:D7/3, 24:D7-1, 25:S6, 26:D7/7, 27:D54

Array{Any,1}[[2006, ((4, "S65/6"), (6, "D7-1"))], [12006, ((6, "S/5"), (6, "D7-1"))], [2006, ((7, "T"), (6, "D7-1"))], [2006, ((8, "Ts"), (6, "D7-1"))], [12006, ((6, "S/3"), (6, "D7-1"))], [12006, ((7, "T/5"), (6, "D7-1"))], [12006, ((8, "D7/5"), (6, "D7-1"))], [12006, ((6, "Dm"), (6, "D7-1"))], [12006, ((7, "T/3"), (6, "D7-1"))], [12006, ((7, "D/5"), (6, "D7-1"))], [2006, ((4, "T-5"), (6, "D7-1"))], [12006, ((8, "D64"), (6, "D7-1"))], [12006, ((7, "D/3"), (6, "D7-1"))], [2006, ((15, "D-5"), (6, "D7-1"))], [12006, ((8, "D7"), (6, "D7-1"))], [12006, ((6, "T64"), (6, "D7-1"))], [12006, ((13, "D7-5"), (6, "D7-1"))], [2006, ((10, "D"), (6, "D7-1"))], [12006, ((6, "Tm"), (6, "D7-1"))], [12006, ((4, "S65"), (6, "D7-1"))], [12006, ((9, "T54"), (6, "D7-1"))], [2006, ((7, "S"), (6, "D7-1"))], [12006, ((8, "D7/3"), (6, "D7-1"))], [2006, ((7, "D7-1"), (6, "D7-1"))], [12006, ((9, "S6"), (6, "D7-1"))], [12006, ((8, "D7/7"), (6, "D7-1"))], [12006, ((7, "D54"), (6, "D7-1"))]]

# # DFS Branch and Bound ALL IN ONE

In [65]:

function prepare(data, functions, function_potential, function_potential_unsorted)
    
    Tonics = ["T", "T/3", "T/5", "T-5"]
    Tonics_dependant = ["T64", "T54"]
    Subdominants = ["S", "S/3", "S/5", "S65", "S65/6", "S6"]
    Dominants = ["D", "D/3", "D/5", "D-5", "D64", "D54", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7"]
    precursor_dominants = ["D", "D-5", "D64", "D54"]
    Mediants = ["Ts", "Tm", "Ss", "Dm"]
    beat_accentuation = Dict("O" => 1, "W" => 2, "S" => 3)
    dominant_order = Dict("D" => 3, "D/3" => 3, "D/5" => 3, "D-5" => 1, "D64" => 1, "D54" => 2, "D7" => 5, "D7-1" => 4, "D7-5" => 4, "D7/3" => 5, "D7/5" => 5, "D7/7" => 5)
    
    start = ["T"]
    leading_end = ["D", "D7-5"]
    final = ["T"]
    
    # Local copy of the original data
    data_local = deepcopy(data)
    # Local copy of dictionary with global function rules : dict[key] = ([1: tonal_map], [2: bass note], [3: double/triple note], [4: Rhythmical rules])
    functions_local = deepcopy(functions)  
    
    # Local copy of list of candidate functions for each timestep [[timestep1], [timestep2], ... ]
    # Each index contains a sorted list of tuples which determine function costs for the given timestep : function_potential_local[i] = [(function_cost, "function_name"), ...]
    function_potential_local = deepcopy(function_potential[1:size(data_local, 2)])
    function_permutations_local = deepcopy(function_permutations)
    
    
    # Create rhythm map for piece
    rhythm_map_local = rhythmify(time_signature, data)
    
    
    # Locate and enforce important events (start, cadences)
    
    start_index = findfirst(x -> x == "S", rhythm_map_local)
    final_index = findlast(x -> x == "S" || x == "W", rhythm_map_local)
    leading_index = findlast(x -> x == "W" || x == "S", rhythm_map_local[1:final_index-1])
    
    filter!(x -> x[2] in start, function_potential_local[start_index])
    for i in leading_index:final_index-1
        filter!(x -> x[2] in precursor_dominants, function_potential_local[i])
        #rhythm_map_local[i] = "D"
    end
    
    filter!(x -> x[2] in leading_end, function_potential_local[final_index-1])
    #rhythm_map_local[final_index-1] = "D"
    for i in final_index:size(data, 2)
        filter!(x -> x[2] in final, function_potential_local[i])
        rhythm_map_local[i] = "A" # Authentic cadence
    end
    
    println("Rhythm map: ", rhythm_map_local)
    
    
    # Prune function_potential by beat restrictions for each chord
    for (i, potential) in enumerate(function_potential_local)
        for (j, (cost, chord)) in enumerate(potential)
            if !(rhythm_map_local[i] in functions_local[chord][4])
                filter!(x -> x[2] != chord, potential)
            end
        end
    end
    
    
    
    #display(function_potential_local)
    
    bestval_func = 1000
    bestval_voices = 1000


    #function_min_list = BuildMinListFunc(function_potential_local, rhythm_map_local, data_local)
    #display(best_function_permutation)
    
    
    initial_solution = []
    done=false
    #initial_solution, bestval_func, bestval_voices, done = BB(function_potential_local, function_potential_unsorted, rhythm_map_local, data_local, function_permutations_local, bestval_func, bestval_voices, initial_solution, true)
    println("Initial solution found")
    if done
        println("Solution found in exploratory search.")
        return initial_solution
    end
    #display(initial_solution)
    println("Initial best function score: ", bestval_func, ", best voices score: ", bestval_voices)
    
    best_solution = BB(function_potential_local, function_potential_unsorted, rhythm_map_local, data_local, function_permutations_local, bestval_func, bestval_voices, initial_solution)

    return best_solution
end

function BB(function_potential_local, function_potential_unsorted, rhythm_map_local, data_local, function_permutations_local, bestval_func, bestval_voices, solution, exploratory=false)
        
    # Initialize global search variables
    best_solution = []
    best_score = 0
    total_depth = size(data_local, 2)
    
    # Value larger than or equal to the largest value in set
    upper_bound = 1000
    # Value smaller than or equal to the smallest value in set
    lower_bound = 0
    
    
    
    # Find a feasible min cost list
    ##### TEST ZONE
    #=
    function_potential_min_list = deepcopy(function_potential_local)
    min_list = [timestep[1][1] for timestep in function_potential_local]
    println(function_potential_min_list[1:total_depth][1])
    =#
    
    println("Building min list...")
    starting_chord_index = 3
    final_chord_index = 3
    
    min_list = buildMinList(function_potential_unsorted[1:16], function_potential_local, rhythm_map_local, starting_chord_index, final_chord_index, false)
    println("Min list built.")
    ##### TEST ZONE
    
    #println(min_list)

    
    
    # Populate active set with all permutations of the possible first functions
    activeset = Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1}, Int64, Int64},1}[]
    score = []
    
    # Func : (cost, function) Find and sort the voicings for the first timestep
    for (cost, func) in function_potential_local[1]
        for potential_voicing in function_permutations_local[func]
            
            temp = [(cost, func, potential_voicing, 0, 0)]
            local_cost = distance_cost(temp, 1, true, false)
            temp = [local_cost, (cost, func, potential_voicing, cost, 0)]
            push!(score, temp)
            
        end
    end

    # Find best starting points
    score = sort!(score, by = x->x[1], rev=true)

    best_score = score[end][1]
    for entry in score
        #if entry[1] == best_score
            start_point = [entry[2]]
            push!(activeset, start_point)
        #end
    end
    display(activeset)
    
    bestval_func = bestval_func
    bestval_voices = bestval_voices
    solution_list = solution
    println(solution_list)
    
    
    
    ###### TestZone
    # push!(function_potential_min_list, (0, 0)) ### REMOVED FOR HEURISTIC TEST
    
    #function_potential_local = *.function_potential_local
    #display(function_potential_local)
    
    function_potential_local[8] = [(0, "D7")]
    function_potential_local[9] = [(0, "Ts")]
    #function_potential_local[11] = [(0, "T")]
    #function_potential_local[12] = [(0, "T")]
    
    display(function_potential_local)
    
    
    
    
    ######
    
    iterations = 0
    pruned = 0
    processed = 0
    solutions = 0
    last_solution = 0

    
    println("Starting search...")
    
    start = time()
    #display(function_potential_min_list)
    
    while !isempty(activeset)
        iterations += 1
        active_node = pop!(activeset)
            
        depth = size(active_node, 1)
        
        # Generate children of active_node and corresponding optimistic bounds
        for (cost, func) in Iterators.reverse(function_potential_local[depth+1]) 
            
            child = deepcopy(active_node)
            temp_child = (cost, func, child[end][3], 0, 0)
            push!(child, temp_child)
            previous_chord_index = getChordIndex(child[end-1][2])
            current_chord_index = getChordIndex(func)
            child_function_cost = function_cost(child, rhythm_map_local)

            child_function_heuristic = min_list[depth+1][current_chord_index, previous_chord_index][1]
            
            deleteat!(child, size(child, 1))
            
            if child_function_cost > 999
                continue
            end
            
            # Child : [(func1_cost, "func1_name", [func1_voicing_list], func1_total_func_cost, func1_total_voice_cost, func1_distance_cost), (function2_cost, "function2_name", , [function2_voicing_list]), ...]
            for voicing in function_permutations_local[func]
                
                child = deepcopy(active_node)
                
                temp_child = (cost, func, voicing, 0, 0)
                push!(child, temp_child)
                processed += 1

                # Find function cost, keep track of cumulative function cost in [4]
                
                
                
                
                # Find voicing cost, keep track of cumulative voicing cost in [5]
                #=
                try
                    child_voicing_cost = voicing_cost(child, rhythm_map_local, false, false)
                catch
                    println(child)
                    return 0
                end
                =#
                child_voicing_cost = voicing_cost(child, rhythm_map_local, false, false)
                child_voicing_heuristic = 2*(total_depth - size(child, 1))

                child_total_cost = child_function_cost + child_function_heuristic + child_voicing_cost + child_voicing_heuristic

                temp_child = (cost, func, voicing, child_function_cost, child_voicing_cost)
                deleteat!(child, size(child, 1))
                push!(child, temp_child)
                
                #println(child)
                #println("Child function Cost: ", child_function_cost)
                #println("Child function heuristic cost: ", child_function_heuristic)

                ## TODO: add cost for poor voicing permutation matchings

                # DEBUG

                if !exploratory && child_total_cost > bestval_func + bestval_voices
                    
                    #println("pruned")

                    deleteat!(child, size(child, 1))
                    pruned += 1     

                elseif exploratory && (child_total_cost > bestval_func + bestval_voices || child_function_cost + child_function_heuristic > 999 || 
                            child_voicing_cost + child_voicing_heuristic > 999)

                    deleteat!(child, size(child, 1))
                    pruned += 1   

                elseif size(child, 1) == total_depth

                    println("depth: ", depth)
                    println(size(child, 1))
                    solutions += 1
                            
                    # Extra full check of candidate solution for illegal combinations
                    #if voicing_cost(child, rhythm_map_local, true) > 999
                        #println("Error in candidate found")
                        #println(child)
                        #continue
                    #end

                    if exploratory
                        solution_list = [(child_total_cost, child)]
                        bestval_func = child_function_cost
                        bestval_voices = child_voicing_cost

                    elseif child_total_cost <= bestval_func + bestval_voices

                        last_solution = iterations

                        println("Current child: ", child)
                        println("function cost: ", child_function_cost + child_function_heuristic)
                        println("child voicing cost: ", child_voicing_cost + child_voicing_heuristic)
                        println("Best yet: ", bestval_func + bestval_voices)

                        if child_total_cost <= bestval_func + bestval_voices
                            bestval_func = child_function_cost
                            bestval_voices = child_voicing_cost
                            pushfirst!(solution_list, (child_total_cost, child))

                        #elseif child_total_cost < bestval_func + bestval_voices
                            #bestval_func = child_function_cost
                            #bestval_voices = child_voicing_cost
                            #solution_list = [(child_total_cost, child)]

                        end
                                        
                        break
                    end
                else
                    push!(activeset, child)
                end
            end
            #println(activeset)
        end
        
        if iterations % 1000 == 0
            println("Iterations done: ", iterations)
            println("Nodes in active set: ", size(activeset, 1))
            println("Current depth: ", depth)
        end     
            *
                
        if exploratory && iterations == 100000
            #try 
                return solution_list, bestval_func, bestval_voices, false
            #catch 
                println("No initial solutions found")
                return 0
            #end
        elseif !exploratory && iterations == 50000 #|| iterations - last_solution == 10000
            println("Timed out")
            #display(activeset)
            println(solution_list)
            break
        end
                        
        ## Maybe add automatic termination if either within 1+a approx of optimistic minimum, after x iterations of no new solutions found, or when crossing a certain efficiency threshold
    end
                
    elapsed = time() - start
    println("Done. Total checked: ", processed, ", Pruned: ", pruned, ", completed: ", solutions, ", time: ", elapsed, ", efficiency: ", (solutions)/elapsed)
    println("Results: bestval_func: ", bestval_func, ", bestval_voices: ", bestval_voices)
    if exploratory
        return solution_list, bestval_func, bestval_voices, true
    else
        return solution_list
    end
end

solution_list = prepare(data[:, 1:16], functions, function_potential, function_potential_unsorted)
            


Rhythm map: ["S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "A", "A"]
Initial solution found
Initial best function score: 1000, best voices score: 1000
Building min list...

16-element Array{Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1},1}:
 [(0, "T", [(53, 1), (68, 3), (72, 5), (77, 1)], 0, 0)]
 [(0, "T", [(41, 1), (60, 5), (68, 3), (77, 1)], 0, 0)]
 [(0, "T", [(53, 1), (60, 5), (68, 3), (77, 1)], 0, 0)]
 [(0, "T", [(53, 1), (65, 1), (72, 5), (80, 3)], 0, 0)]
 [(0, "T", [(41, 1), (56, 3), (60, 5), (65, 1)], 0, 0)]
 [(0, "T", [(53, 1), (56, 3), (60, 5), (65, 1)], 0, 0)]
 [(0, "T", [(41, 1), (56, 3), (65, 1), (72, 5)], 0, 0)]
 [(0, "T", [(53, 1), (56, 3), (65, 1), (72, 5)], 0, 0)]
 [(0, "T", [(41, 1), (48, 5), (56, 3), (65, 1)], 0, 0)]
 [(0, "T", [(41, 1), (60, 5), (65, 1), (68, 3)], 0, 0)]
 [(0, "T", [(53, 1), (60, 5), (65, 1), (68, 3)], 0, 0)]
 [(0, "T", [(53, 1), (65, 1), (68, 3), (72, 5)], 0, 0)]
 [(0, "T", [(41, 1), (53, 1), (60, 5), (68, 3)], 0, 0)]
 [(0, "T", [(53, 1), (53, 1), (60, 5), (68, 3)], 0, 0)]
 [(0, "T", [(41, 1), (53, 1), (56, 3), (60, 5)], 0, 0)]
 [(0, "T", [(53, 1), (53, 1), (56, 3), (60, 5)], 0, 0)]

16-element Array{Array{Tuple{Int64,String},1},1}:
 [(0, "T")]                                                                                                                                                                                                                                        
 [(2, "D7-5"), (2, "Ts"), (3, "S"), (3, "S/3"), (3, "S/5"), (3, "S6"), (4, "D-5"), (4, "D7"), (4, "D7/3"), (4, "D7/5")  …  (4, "T/3"), (4, "T/5"), (4, "Tm"), (5, "D"), (5, "D/3"), (5, "D/5"), (5, "T54"), (6, "D7-1"), (7, "T"), (10, "T-5")]    
 [(0, "Tm"), (2, "D"), (2, "D/3"), (2, "D/5"), (2, "D7"), (2, "D7/3"), (2, "D7/5"), (2, "D7/7"), (2, "T"), (2, "T/3")  …  (3, "Ts"), (4, "D7-1"), (4, "S65"), (4, "T54"), (5, "D-5"), (5, "S"), (5, "S/3"), (5, "S/5"), (7, "S6"), (7, "T-5")]     
 [(0, "Ts"), (4, "D54"), (5, "D"), (5, "D/3"), (5, "D/5"), (5, "S65"), (5, "T/3"), (5, "T/5"), (6, "T"), (7, "D7")  …  (7, "Tm"), (8, "D-5"), (8, "D7-5"), (8, "S"), (8, "S/3"), (8, "S/5"), (9, "T-5"), (10, "S6"), (11, 


Min list built.
Any[]
Starting search...
depth: 15
16
Current child: Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64}[(0, "T", [(53, 1), (53, 1), (56, 3), (60, 5)], 0, 0), (2, "D7-5", [(48, 1), (58, 7), (60, 1), (64, 3)], 2, 6), (2, "T", [(53, 1), (56, 3), (60, 5), (65, 1)], 4, 12), (0, "Ts", [(49, 1), (61, 1), (65, 3), (68, 5)], 4, 19), (2, "D7", [(48, 1), (58, 7), (67, 5), (76, 3)], 6, 22), (6, "Ts", [(49, 1), (56, 5), (65, 3), (77, 3)], 12, 23), (4, "D7-1", [(55, 5), (58, 7), (70, 7), (76, 3)], 16, 27), (0, "D7", [(48, 1), (58, 7), (67, 5), (76, 3)], 16, 32), (0, "Ts", [(49, 1), (56, 5), (65, 3), (77, 3)], 16, 33), (2, "S", [(46, 1), (58, 1), (65, 5), (73, 3)], 18, 38), (1, "S/3", [(49, 3), (65, 5), (70, 1), (70, 1)], 19, 47), (7, "D/3", [(52, 3), (60, 1), (72, 1), (79, 5)], 26, 57), (6, "D", [(48, 1), (60, 1), (67, 5), (76, 3)], 32, 65), (6, "D", [(48, 1), (60, 1), (67, 5), (76, 3)], 38, 67), (7, "T", [(53, 1), (60, 5), (68, 3), (77, 1)], 45, 71), (3, "T", [(53, 1), (56

8-element Array{Any,1}:
 (76, Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64}[(0, "T", [(53, 1), (53, 1), (56, 3), (60, 5)], 0, 0), (2, "Ts", [(49, 1), (56, 5), (61, 1), (65, 3)], 2, 6), (2, "D7", [(48, 1), (58, 7), (64, 3), (67, 5)], 4, 7), (0, "Ts", [(49, 1), (56, 5), (65, 3), (65, 3)], 4, 8), (2, "S65", [(46, 1), (55, 6), (61, 3), (65, 5)], 6, 13), (3, "T/5", [(48, 5), (56, 3), (60, 5), (65, 1)], 9, 14), (4, "S/3", [(49, 3), (58, 1), (58, 1), (65, 5)], 13, 14), (0, "D7", [(48, 1), (58, 7), (64, 3), (67, 5)], 13, 16), (0, "Ts", [(49, 1), (56, 5), (65, 3), (65, 3)], 13, 17), (2, "S/3", [(49, 3), (58, 1), (65, 5), (70, 1)], 15, 19), (5, "T/5", [(48, 5), (60, 5), (65, 1), (68, 3)], 20, 20), (4, "S65", [(46, 1), (61, 3), (65, 5), (67, 6)], 24, 20), (6, "D", [(48, 1), (52, 3), (60, 1), (67, 5)], 30, 23), (6, "D", [(48, 1), (52, 3), (60, 1), (67, 5)], 36, 25), (7, "T", [(41, 1), (48, 5), (56, 3), (65, 1)], 43, 26), (3, "T", [(41, 1), (48, 5), (56, 3), (65, 1)], 46, 27)])   
 (8

In [67]:

for chord in solution_list[1]
    display(chord)
end

76

16-element Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1}:
 (0, "T", [(53, 1), (53, 1), (56, 3), (60, 5)], 0, 0)    
 (2, "Ts", [(49, 1), (56, 5), (61, 1), (65, 3)], 2, 6)   
 (2, "D7", [(48, 1), (58, 7), (64, 3), (67, 5)], 4, 7)   
 (0, "Ts", [(49, 1), (56, 5), (65, 3), (65, 3)], 4, 8)   
 (2, "S65", [(46, 1), (55, 6), (61, 3), (65, 5)], 6, 13) 
 (3, "T/5", [(48, 5), (56, 3), (60, 5), (65, 1)], 9, 14) 
 (4, "S/3", [(49, 3), (58, 1), (58, 1), (65, 5)], 13, 14)
 (0, "D7", [(48, 1), (58, 7), (64, 3), (67, 5)], 13, 16) 
 (0, "Ts", [(49, 1), (56, 5), (65, 3), (65, 3)], 13, 17) 
 (2, "S/3", [(49, 3), (58, 1), (65, 5), (70, 1)], 15, 19)
 (5, "T/5", [(48, 5), (60, 5), (65, 1), (68, 3)], 20, 20)
 (4, "S65", [(46, 1), (61, 3), (65, 5), (67, 6)], 24, 20)
 (6, "D", [(48, 1), (52, 3), (60, 1), (67, 5)], 30, 23)  
 (6, "D", [(48, 1), (52, 3), (60, 1), (67, 5)], 36, 25)  
 (7, "T", [(41, 1), (48, 5), (56, 3), (65, 1)], 43, 26)  
 (3, "T", [(41, 1), (48, 5), (56, 3), (65, 1)], 4

# # Collect solution voicings and create piano roll

In [66]:
## Convert back to piano roll

function makePianoRoll(notes, name, i)
    println(size(notes, 1))
    pianoroll = zeros(Int8, 128, size(notes,1))
    counter = 1
    for set in notes[:]
        for voice in set
            #println(voice)
            pianoroll[voice[1], counter] = 1
        end
        counter = counter + 1
        #println(pianoroll[:,1])
    end
    #println(findall(x->x==1, pianoroll))
    empty = zeros(Int8, 1, size(notes, 1))
    new = vcat(pianoroll, empty)
    filepath = "output\\$name$i.csv"
    println(filepath)
    writedlm(filepath,  new, ',')
end

solution_notes_list = []
solution_functions_list = []

for i in 1:10

    push!(solution_notes_list, [timestep[3] for timestep in solution_list[i][2]])
    push!(solution_functions_list, [timestep[2] for timestep in solution_list[i][2]])
    
    makePianoRoll(solution_notes_list[i], "test", i)
end

display(solution_functions_list)

#makePianoRoll(solution_notes[1], "test", i)

println(solution_functions_list[1])




16
output\test1.csv
16
output\test2.csv
16
output\test3.csv
16
output\test4.csv
16
output\test5.csv
16
output\test6.csv
16
output\test7.csv
16
output\test8.csv


BoundsError: BoundsError: attempt to access 8-element Array{Any,1} at index [9]

In [31]:
hm = [(10, "T", [(48, 1), (52, 3), (55, 5), (60, 1)], 0, 0, 3), 
    (0, "S6", [(41, 1), (53, 1), (62, 6), (69, 3)], 10, 6, 5), 
    (2, "T", [(48, 1), (60, 1), (64, 3), (67, 5)], 12, 13, 7), 
    (4, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 16, 19, 10), 
    (2, "D64", [(43, 1), (55, 1), (60, 4), (64, 6)], 18, 23, 13), 
    (8, "D", [(43, 1), (55, 1), (59, 3), (62, 5)], 26, 24, 17), 
    (4, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 30, 28, 19), 
    (0, "S", [(53, 1), (53, 1), (60, 5), (69, 3)], 30, 34, 21), 
    (4, "D/3", [(47, 3), (55, 1), (67, 1), (74, 5)], 34, 42, 23), 
    (4, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 38, 44, 25), 
    (4, "Ts", [(45, 1), (64, 5), (69, 1), (72, 3)], 42, 50, 27), 
    (6, "D/3", [(47, 3), (62, 5), (67, 1), (74, 5)], 48, 51, 28), 
    (18, "D7/3", [(59, 3), (62, 5), (67, 1), (77, 7)], 66, 56, 28)]
println(size(hm, 1))

13


In [32]:
hm = [(10, "T", [(48, 1), (52, 3), (55, 5), (60, 1)], 0, 0, 3), 
    (0, "S6", [(41, 1), (53, 1), (62, 6), (69, 3)], 10, 6, 5), 
    (2, "T", [(48, 1), (60, 1), (64, 3), (67, 5)], 12, 13, 7), 
    (4, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 16, 19, 10), 
    (2, "D64", [(43, 1), (55, 1), (60, 4), (64, 6)], 18, 23, 13), 
    (8, "D", [(43, 1), (55, 1), (59, 3), (62, 5)], 26, 24, 17), 
    (4, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 30, 28, 19), 
    (0, "S", [(53, 1), (53, 1), (60, 5), (69, 3)], 30, 34, 21), 
    (4, "D/3", [(47, 3), (55, 1), (67, 1), (74, 5)], 34, 42, 23), 
    (4, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 38, 44, 25), 
    (4, "Ts", [(45, 1), (64, 5), (69, 1), (72, 3)], 42, 50, 27), 
    (6, "D/3", [(47, 3), (62, 5), (67, 1), (74, 5)], 48, 51, 28), 
    (18, "D7/3", [(59, 3), (62, 5), (67, 1), (77, 7)], 66, 56, 28)]
println(size(hm, 1))

13


In [33]:
wat = [(10, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), 
    (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4), 
    (2, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10), 
    (4, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16), 
    (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24), 
    (6, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27), 
    (4, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37), 
    (8, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38), 
    (4, "S/3", [(45, 3), (53, 1), (65, 1), (72, 5)], 38, 40), 
    (2, "Tm", [(52, 1), (55, 3), (67, 3), (71, 5)], 40, 43), 
    (6, "S", [(53, 1), (57, 3), (60, 5), (65, 1)], 46, 45), 
    (8, "D54", [(55, 1), (55, 1), (60, 4), (62, 5)], 54, 46)]

best = (146.0, [(10, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), 
        (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4), 
        (2, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10), 
        (4, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16), 
        (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24), 
        (6, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27), 
        (4, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37), 
        (8, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38), 
        (4, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42), 
        (2, "D/3", [(47, 3), (50, 5), (55, 1), (67, 1)], 40, 44), 
        (8, "T", [(48, 1), (52, 3), (60, 1), (67, 5)], 48, 45), 
        (6, "S", [(53, 1), (57, 3), (60, 5), (65, 1)], 54, 49), 
        (18, "D-5", [(55, 1), (55, 1), (59, 3), (67, 1)], 72, 49), 
        (6, "D7", [(55, 1), (59, 3), (62, 5), (65, 7)], 78, 52), 
        (10, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 88, 53), 
        (6, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 94, 52)]) 
println(size(wat, 1))

12


In [34]:
empty = zeros(Int8, 1, size(data, 2))
input_fixed = deepcopy(data)
new = vcat(input_fixed, empty)
writedlm("input\\dummyMajorMultiple.csv",  new, ',')

# # Construct a naive solution

# # NODAL

In [35]:
import Optim
SimulatedAnnealing(; neighbor = default_neighbor!,
                    T = default_temperature,
                    p = kirkpatrick)

UndefVarError: UndefVarError: default_neighbor! not defined